<a href="https://colab.research.google.com/github/gabyakcelik/street_psych/blob/master/St_Psych_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [ ]:
import numpy as np
from scipy.stats import pearsonr
from sklearn import datasets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler 
from statsmodels.iolib.summary2 import summary_col
from statsmodels.api import add_constant
from sklearn.decomposition import PCA

# Functions

In [ ]:
def standardize_data(data):
    scaler = StandardScaler()
    standardized = scaler.fit_transform(data)
    df_standardized = pd.DataFrame(standardized)
    return df_standardized


def clean_lo_data(file_path):
  data = pd.read_csv(file_path)
  data = pd.DataFrame.sort_index(data, axis=0)
  cleaned_lo_data = standardize_data(data)
  cleaned_lo_data.columns = ['ED', 'Hue', 'Sat','Lum', 'sdHue', 'sdSat', 'sdBright',
'Entropy', 'StraightED', 'NonStraightED']
  return cleaned_lo_data

def clean_mid_data(file_path):
  data = pd.read_csv(file_path, index_col=0)
  data = pd.DataFrame.sort_index(data, axis=0)
  cleaned_mid_data = standardize_data(data)
  cleaned_mid_data.columns=['Road', 'Sidewalk', 'Building', 'Wall','Fence', 'Pole', 'Traffic Light', 'Traffic Sign', 'Vegetation','Terrain', 'Sky',
'Person', 'Rider', 'Car','Truck', 'Bus', 'Train','Motorcycle','Bicycle', 'Unlabeled']
  return cleaned_mid_data

def clean_rates_data(file_path):
  data = pd.read_csv(file_path, index_col=0)
  data = data.drop(['Humanscale', 'Order'], axis=1)
  cleaned_data = data.rename(columns={"Order2" : "Disorder", "Humanscale2" : "Humanscale"})
  cleaned_rates_data = standardize_data(cleaned_data)
  cleaned_rates_data.columns=["Walkability", "Preference", "Imageability", "Complexity", "Enclosure", "Transparency", "Humanscale", "Disorder"]
  return cleaned_rates_data

def clean_data(file_path):
  data = pd.read_csv(file_path)
  cleaned_data = standardize_data(data)
  return cleaned_data

def clean_data_0(file_path):
  data = pd.read_csv(file_path, index_col=0)
  cleaned_data = standardize_data(data)
  return cleaned_data


def combine_ratings_and_features(lowleveldata, midleveldata, ratingsdata):
    all_feat = pd.concat([lowleveldata, midleveldata], axis=1)
    all_feat.rename(columns={'const': 'Constant'})
    all_feat_ratings = pd.concat([all_feat, ratingsdata], axis=1)
    return all_feat_ratings


def run_reg(rating, data):
    reg = sm.OLS(rating, data).fit()
    return reg.summary()


def create_reg_table(*args):
    reg_list = []
    for a in len(args):
        reg_list.append(a)
    reg_table = summary_col([reg_list], stars=True, float_format='%0.2f')
    return reg_table


def corrfunc(x, y, ax=None, **kws):
    """Plot the correlation coefficient in the top left corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    corrcoeff = 'r'
    ax.annotate(f'{corrcoeff} = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)


def create_ratings_corrgram(ratings):
    """creating a correlation matrix for all perceptual qualities."""
    corrgram_qualities = sns.pairplot(ratings, kind='reg', diag_kind="kde",
                                      plot_kws={'line_kws': {'color': 'black'}, 'scatter_kws': {'alpha': 0.5}})
    corrgram_qualities.set(xlim=(0.0, 1.0), ylim=(0.0, 1.0))
    corrgram_qualities.map_upper(corrfunc)
    corrgram_qualities.map_lower(corrfunc)
    corrgram_qualities.map_diag(sns.distplot)
    return plt.show()


def create_ratings_and_feat_corrgram(all_feat_ratings):
    """creating a correlation matrix for all perceptual qualities and features."""
    sns.set(style="white")
    f, ax = plt.subplots(figsize=(40, 40))
    sns.set(font_scale=1.99)
    sns.heatmap(all_feat_ratings.corr(), annot=True, fmt='.01g', vmin=-1, vmax=1, center=0,
                          annot_kws={"size": 17})
    return ax.tick_params(labelsize=27)


def standardized_low_boxplot(df_standardized):
    f, ax = plt.subplots(figsize=(25, 15))
    sns.boxplot(data=df_standardized)
    ax.tick_params(labelsize=22)
    #plt.title("Visual Features - Standardized",
    #        fontsize=40)  # You can comment this line out if you don't need title
    return plt.show()


def standardized_mid_boxplot(df_standardized):
    sns.set(font_scale=1.99)
    plt.figure(figsize=(20,17))
    g = sns.boxplot(data = df_standardized)
    #ax.tick_params(labelsize=90)
   # plt.title("Mid-Level Visual Features - Standardized", fontsize=40) 
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    plt.show()


# Cleaning Data

## Clean Sidewalk Data

### Low-Level

In [ ]:
clean_lo_sw = clean_lo_data("/content/drive/My Drive/street_psych/Streetviewimages_avFeaturesNofix1.csv")
clean_lo_sw = clean_lo_sw.head(552) #using the first 552 images only
clean_lo_sw

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED
0,-0.697388,0.582741,-0.829933,0.965938,1.020501,-1.769262,1.537084,0.988298,-1.645022,0.580355
1,0.142487,0.244800,1.469235,0.135219,-0.683353,1.410906,-0.049747,0.706928,0.197814,-0.011075
2,1.043314,-2.309889,-1.146498,-0.854800,0.088301,-0.794455,-0.971109,-0.106545,2.490735,-1.105760
3,0.496954,-0.299344,0.456589,0.652278,0.120898,1.793095,1.255978,0.477388,1.035100,-0.341145
4,-1.501323,0.031841,-0.651723,1.288547,1.839265,-0.115493,0.271976,-0.040704,-0.620530,-0.227754
...,...,...,...,...,...,...,...,...,...,...
547,-0.601862,1.133674,-0.182982,0.640171,1.050453,-0.545005,-0.646316,0.035079,-0.142850,-0.416623
548,-1.122581,0.485214,-1.280342,1.756527,1.128100,-0.687161,0.638593,0.102449,-0.688028,-0.202492
549,-0.286593,0.009437,-0.624113,0.086371,0.142504,-0.198892,0.065347,1.381106,0.761069,-0.806091
550,-0.766012,0.261837,-1.473116,1.839824,0.420205,-0.645071,0.151189,0.014913,-0.113812,-0.389604


### Mid-Level

In [ ]:
clean_mid_sw = clean_mid_data("/content/drive/My Drive/street_psych/midlvlsidewalk_new.csv")
clean_mid_sw = clean_mid_sw.head(552) #using the first 552 images only
clean_mid_sw

,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,1.606022,-0.390942,-0.677097,-0.190748,-0.675653,0.087769,-0.335653,0.021891,-0.403206,-0.539987,1.201695,-0.026431,0.561862,-0.626202,-0.216990,-0.125748,-0.042145,-0.046379,-0.093707,-0.574748
1,0.079553,1.598872,-0.161347,-0.190748,-0.367752,-0.369828,-0.374001,-0.547766,0.160267,-0.403506,-0.084766,1.054426,-0.145018,-0.597545,0.129925,-0.130470,-0.042145,-0.088317,0.659986,-0.041967
2,-0.672936,-0.398976,1.465504,-0.190748,-0.212524,-0.491069,-0.403608,-0.441761,0.045671,-0.297560,-1.195210,0.658537,-0.145018,0.916010,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.599589
3,0.804310,-0.935804,0.180557,-0.190748,-0.604152,-0.605742,-0.405800,-0.464203,0.614335,-0.339743,-0.421392,-0.169500,-0.145018,-0.110990,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.589360
4,1.493925,-0.897668,-1.291167,-0.167972,0.044460,-0.486964,-0.400332,-0.102259,-0.603201,-0.570708,2.104099,-0.313549,-0.145018,-0.253864,-0.090894,0.165332,-0.042145,-0.088317,-0.247871,-0.361748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,0.151127,-0.908833,-0.550650,-0.187266,-0.268936,-0.235177,1.139660,0.210504,-0.872190,-0.606905,1.300611,0.472351,-0.145018,1.860915,-0.198197,-0.130470,-0.042145,-0.088317,0.594731,0.018056
548,1.640049,-1.083393,-0.971368,-0.190748,-0.697320,-0.429765,-0.317020,-0.744973,-0.793493,-0.599387,1.895585,0.081361,-0.145018,-0.488606,1.682480,-0.130470,-0.042145,-0.088317,-0.246240,0.148328
549,-1.441450,0.498869,1.073859,0.398207,1.066128,0.569176,-0.223853,0.779206,-1.123328,-0.607091,-0.309367,3.410656,-0.145018,1.464621,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,0.269947
550,1.932614,-0.922084,-0.619306,-0.167570,-0.836452,-0.281155,-0.406898,0.184241,-0.824157,-0.523930,1.667382,0.320952,-0.145018,-0.675682,-0.219113,-0.130470,-0.042145,-0.088317,-0.247871,-0.581155


### Ratings

In [ ]:
clean_rates_sw = clean_rates_data("/content/drive/My Drive/street_psych/scoreTable_2.csv")
clean_rates_sw

,Walkability,Preference,Imageability,Complexity,Enclosure,Transparency,Humanscale,Disorder
0,-0.328069,-0.396165,-0.338632,-0.538060,-0.332572,0.167202,0.599738,-0.738702
1,1.841204,2.017750,2.287950,2.898411,-0.820297,1.115442,-1.113798,0.084024
2,-0.006695,-0.038547,-0.229192,0.934713,0.837968,1.984662,-0.314148,-0.224498
3,1.118113,1.481325,1.521864,2.014747,0.252698,1.589562,-0.085677,-0.430180
4,-0.810130,-0.217356,-0.448073,0.934713,-1.210477,-0.860058,-1.113798,0.803909
...,...,...,...,...,...,...,...,...
547,-0.729786,-0.664377,-0.995278,1.327453,-1.405567,0.167202,-1.799213,1.523795
548,-0.970817,-1.111399,-1.433042,-0.734430,-1.112932,-0.860058,-0.199913,-0.533021
549,-1.211847,-1.290207,-0.557514,0.836528,-0.625207,1.431522,0.713973,2.140840
550,-0.970817,-0.843186,-1.214160,-1.028985,-0.235027,-0.227898,-0.885327,-0.430180


##Clean Road Data


### Low-Level

In [ ]:
clean_lo_rd = clean_lo_data("/content/drive/My Drive/street_psych/Roadviewimages_avFeaturesNofix1.csv")
clean_lo_rd = clean_lo_rd.head(552) #using the first 552 images only
clean_lo_rd

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED
0,0.082438,0.550033,0.790950,-0.198505,-0.070003,0.153939,0.129149,0.967512,0.161195,-0.091117
1,-0.788018,1.002600,0.438174,-0.185384,1.546090,-0.193184,-0.979389,-1.424131,-0.621514,-0.139558
2,-0.157772,-2.151727,-0.683065,-0.138796,-0.468528,-0.278418,0.824622,1.271180,0.667325,-0.710098
3,-0.697148,-1.906191,-0.246839,0.768918,0.729539,0.294893,1.456231,0.314191,0.335352,-0.468591
4,-0.830773,0.180614,0.770656,0.015852,0.949692,1.130193,-0.703302,-0.201237,-0.471402,-0.370473
...,...,...,...,...,...,...,...,...,...,...
547,-0.707258,1.350689,-0.401609,-0.071152,0.323305,-0.841767,-0.747521,0.160509,0.843873,-1.247143
548,-1.431420,0.434880,-1.619145,1.646230,-0.174660,-0.852298,-0.039796,-0.377581,-0.548982,0.537702
549,-0.888877,0.341988,-1.034897,0.982797,0.204718,-0.842375,-0.132077,-0.333057,-0.459048,0.106841
550,-0.215875,0.249791,-1.115426,0.442784,-0.385735,-0.516379,0.047866,0.799870,0.292311,0.217916


### Mid-Level

In [ ]:
clean_mid_rd = clean_mid_data("/content/drive/My Drive/street_psych/mid_level_output_road.csv")
clean_mid_rd = clean_mid_rd.head(552) #using the first 552 images only
clean_mid_rd

,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,-0.311138,1.531860,-0.835667,-0.017142,-0.739186,-0.216989,-0.425122,2.940363,0.362426,0.526167,0.146232,-0.358936,-0.129245,-0.619744,1.939384,-0.064760,-0.042145,-0.068924,-0.240763,-0.104711
1,0.864339,-0.300955,0.206693,-0.147998,-0.774902,-0.560783,-0.391109,-0.236009,-0.524059,-0.508972,0.805189,-0.042003,-0.129245,-0.719019,-0.322795,-0.166632,-0.042145,-0.068924,-0.240763,-0.286816
2,-0.580077,0.206287,0.991756,-0.147998,-0.835441,-0.279497,-0.218089,-0.580116,-0.593476,-0.653270,0.228581,-0.358936,-0.129245,0.622919,-0.053923,-0.163850,-0.042145,-0.068924,-0.240763,-0.221381
3,1.077610,-0.273756,-0.369115,-0.147998,0.874103,1.866932,-0.185556,-0.299536,-0.166820,-0.393211,0.329094,-0.266795,-0.129245,-0.721203,-0.299559,-0.152029,-0.042145,-0.068924,-0.034081,-0.251799
4,0.846085,-0.985651,-0.882177,-0.147998,-0.627931,0.078295,-0.426600,-1.017245,-0.299888,-0.635736,0.983316,-0.356553,-0.129245,0.729099,-0.326778,-0.152377,-0.042145,-0.068924,-0.240763,0.555692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,-0.568511,-0.688446,-0.150352,-0.147998,-0.756687,-0.811140,1.186773,-0.576334,-0.738174,-0.653270,0.849797,2.378285,-0.129245,2.754196,-0.251096,-0.166632,-0.042145,-0.068924,1.233207,-0.197660
548,1.008263,-0.232674,-0.264446,0.155987,0.548552,-0.424047,-0.426600,0.690433,-1.112475,-0.648618,1.363411,-0.358936,-0.129245,-0.226308,-0.323459,-0.166632,-0.042145,-0.068924,-0.240763,-0.105356
549,-0.021933,0.987030,1.030121,-0.147998,-0.721328,-0.529529,-0.426600,-0.131642,-1.181106,-0.653270,0.584835,1.533924,-0.129245,-0.170012,-0.327110,-0.166632,-0.042145,-0.068924,-0.180934,-0.189672
550,-0.336835,1.060977,1.247690,-0.109568,0.558374,1.745498,-0.426600,2.682472,-1.183465,-0.653270,-0.003596,0.288433,-0.129245,0.067712,-0.325451,-0.166632,-0.042145,-0.068924,-0.240763,-0.258254


### Ratings

In [ ]:
clean_rates_rd = clean_rates_data("/content/drive/My Drive/street_psych/scoreTable_2.csv")
clean_rates_rd 

,Walkability,Preference,Imageability,Complexity,Enclosure,Transparency,Humanscale,Disorder
0,-0.328069,-0.396165,-0.338632,-0.538060,-0.332572,0.167202,0.599738,-0.738702
1,1.841204,2.017750,2.287950,2.898411,-0.820297,1.115442,-1.113798,0.084024
2,-0.006695,-0.038547,-0.229192,0.934713,0.837968,1.984662,-0.314148,-0.224498
3,1.118113,1.481325,1.521864,2.014747,0.252698,1.589562,-0.085677,-0.430180
4,-0.810130,-0.217356,-0.448073,0.934713,-1.210477,-0.860058,-1.113798,0.803909
...,...,...,...,...,...,...,...,...
547,-0.729786,-0.664377,-0.995278,1.327453,-1.405567,0.167202,-1.799213,1.523795
548,-0.970817,-1.111399,-1.433042,-0.734430,-1.112932,-0.860058,-0.199913,-0.533021
549,-1.211847,-1.290207,-0.557514,0.836528,-0.625207,1.431522,0.713973,2.140840
550,-0.970817,-0.843186,-1.214160,-1.028985,-0.235027,-0.227898,-0.885327,-0.430180


##Clean Emily Talen Data

### Low-Level

In [ ]:
clean_lo_et = clean_lo_data("/content/drive/My Drive/street_psych/Emily_Images_avFeaturesNofix1.csv")
clean_lo_et

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED
0,1.265458,-0.851859,-0.270888,0.372972,0.158647,-0.011795,-0.293687,0.853004,-0.491502,1.402001
1,0.091315,1.524110,-0.156125,-0.116810,1.790921,-0.035404,0.548584,1.519825,0.769589,-0.219329
2,-0.525075,1.727045,-0.176056,0.641278,-0.557655,-0.218428,2.263158,1.036623,-0.680173,0.497248
3,0.234903,-1.671054,0.576315,-0.883231,-1.051611,1.093536,2.112478,0.209921,-0.145027,-0.322996
4,-0.778060,2.022104,0.457735,0.573101,-0.052567,0.645432,2.083170,0.981226,0.352090,-1.034856
...,...,...,...,...,...,...,...,...,...,...
1114,2.535470,0.381666,-0.908047,0.016100,-0.001855,-0.976698,-0.542233,0.927218,-1.704212,2.669128
1115,1.910454,0.858489,-0.937488,0.081708,-0.385447,-0.725969,1.123742,0.018588,-1.507824,2.410881
1116,1.713471,0.498388,-0.414334,-0.727826,-0.295367,-0.232528,-0.537495,1.300442,-0.921338,2.269649
1117,-0.153634,0.588014,-0.766334,0.880246,-0.617186,-0.680503,-0.210218,-0.090134,-0.694914,0.039262


### Mid-Level

In [ ]:
clean_mid_et = clean_mid_data("/content/drive/My Drive/street_psych/EmilyImages_object_ratios.csv")
clean_mid_et

,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,-1.182648,2.132680,-1.585717,-0.191013,0.364651,1.029592,-0.574475,-0.282861,1.584209,0.869271,-0.260070,0.281507,-0.168500,0.769642,-0.307147,-0.081179,-0.045038,-0.096961,-0.223434,-0.162660
1,-0.629600,-0.853475,-0.012611,-0.191013,-0.628944,-0.061473,-0.674496,0.456005,0.316914,-0.382630,-0.245267,-0.377845,-0.168500,3.429470,-0.087651,-0.257506,-0.045038,-0.096961,-0.223434,-0.254883
2,-0.391237,-0.476075,-1.410169,-0.091038,0.401927,-0.937853,-0.754019,-0.690603,0.538499,0.129382,1.272925,-0.530266,-0.168500,-0.763071,0.385831,-0.239085,-0.045038,-0.096961,-0.223434,1.005093
3,-0.798453,-0.834788,-0.781849,-0.191013,1.794002,-0.660562,-0.744084,-0.729229,0.529481,-0.234995,0.254465,-0.057072,-0.168500,-0.866147,0.414265,-0.257506,-0.045038,-0.096961,-0.223434,1.896127
4,-0.862211,-1.065663,-0.318065,-0.154715,-0.253409,1.820911,1.593102,0.009532,-0.750045,-0.380199,0.540548,-0.499765,-0.168500,0.654618,-0.400614,-0.257506,-0.045038,-0.096961,-0.223434,1.597656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,-0.070108,-0.597025,-0.870212,-0.191013,-0.653160,-1.020278,-0.598415,-0.603192,1.791928,6.863127,-0.843106,-0.445238,-0.168500,1.382717,-0.400614,-0.257506,-0.045038,1.646961,0.998227,-0.346011
1115,0.908227,-1.088441,-1.377703,1.427248,-0.272184,-1.326267,-0.751377,-0.300534,3.413975,-0.356800,-0.354866,-0.529243,-0.168500,0.399087,-0.400614,-0.257506,-0.045038,-0.096961,-0.223434,-0.649815
1116,1.654465,-0.629249,-0.889648,-0.191013,0.777099,-1.280959,-0.715072,-0.713779,2.602343,-0.192530,-1.100384,-0.512033,-0.168500,0.679978,-0.271558,-0.257506,-0.045038,-0.096961,-0.212060,-0.627752
1117,2.390380,0.092292,-0.713610,-0.191013,-0.077320,0.353920,-0.371189,1.661664,0.456971,-0.366943,-0.422818,-0.133240,0.157199,-0.420575,-0.397684,0.726936,-0.045038,-0.096961,0.061209,-0.473040


### Ratings

In [ ]:
rates_et = pd.read_csv("/content/drive/My Drive/street_psych/scoreTable_emily.csv", index_col=0)
clean_rates_et = standardize_data(rates_et)
clean_rates_et.columns=["Walkability", "Preference", "Imageability", "Complexity", "Enclosure", "Transparency", "Humanscale", "Disorder"]
clean_rates_et

,Walkability,Preference,Imageability,Complexity,Enclosure,Transparency,Humanscale,Disorder
0,-1.083078,0.327144,-0.721013,0.447036,-0.715858,-1.116452,0.781183,0.970461
1,0.900115,0.459700,0.411867,1.957607,0.197174,1.425430,-1.070382,1.586034
2,-1.477525,-0.854817,-1.051695,-1.361834,-1.014728,-1.488415,-0.427478,-0.828639
3,-0.091629,-0.496312,-0.506069,-0.843381,-0.442441,0.545975,0.091711,-0.558512
4,-0.752251,-0.197558,0.380574,-0.815531,-0.472820,-0.160885,0.413035,0.135631
...,...,...,...,...,...,...,...,...
1114,0.798999,-0.364145,0.138762,-0.159967,0.016452,0.296189,0.291880,-0.760207
1115,-0.031218,1.103546,-0.237680,-0.125628,-0.770221,-1.483536,1.086838,0.152220
1116,0.508313,0.327144,-0.033193,0.156313,0.249540,-1.034702,1.978154,0.671602
1117,0.112988,0.209316,-0.033193,0.293368,-0.745416,-0.073899,1.510053,-1.115314


## Clean Img307/Omid's Data

### Low-Level

In [ ]:
lo_ok = pd.read_csv("/content/drive/My Drive/street_psych/omidsimageslowlvl_new.csv")
clean_lo_ok = standardize_data(lo_ok)
clean_lo_ok.columns = ['ED', 'Hue', 'Sat','Lum', 'sdHue', 'sdSat', 'sdBright',
'Entropy', 'StraightED', 'NonStraightED']
clean_lo_ok

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED
0,-1.232490,-0.082796,0.119985,1.617739,-0.497826,0.820966,-0.950572,0.814664,1.185487,-1.412920
1,-0.617268,-0.427681,-1.478633,-0.497113,1.166007,-0.206832,-0.572974,-0.495379,1.508678,-1.249677
2,-0.113970,-0.090056,-1.625161,0.802979,-0.230390,-0.739948,-1.476310,0.405306,0.007830,-0.371388
3,1.550131,0.652147,-0.361839,-0.691105,-0.119287,-0.383548,-0.880278,0.794122,-0.997045,1.341663
4,2.226447,0.308501,1.523688,-0.232258,-1.580803,0.470686,0.352914,1.316032,0.178628,1.383540
...,...,...,...,...,...,...,...,...,...,...
302,-1.081626,0.728927,-1.460687,-0.377715,0.385941,-0.457906,1.194757,0.361908,-0.709314,-0.552956
303,0.676321,0.376195,0.719431,-0.969375,-0.622591,1.309998,-0.968416,0.767026,0.340145,0.297682
304,0.814262,0.221237,-0.319040,-0.927605,-0.916105,0.016698,-0.417879,0.924877,-0.964916,1.020624
305,1.503559,0.279991,-0.061190,0.192838,-1.692653,-1.070291,-2.670133,-1.034059,-0.649023,1.168845


### Mid-Level

In [ ]:
mid_ok = pd.read_csv("/content/drive/My Drive/street_psych/images307_object_ratios.csv", index_col=0)
clean_mid_ok = standardize_data(mid_ok)
clean_mid_ok.columns=['Road', 'Sidewalk', 'Building', 'Wall','Fence', 'Pole', 'Traffic Light', 'Traffic Sign', 'Vegetation','Terrain', 'Sky',
'Person', 'Rider', 'Car','Truck', 'Bus', 'Train','Motorcycle','Bicycle', 'Unlabeled']
clean_mid_ok

,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,-0.449545,-0.576600,-0.115634,-0.165720,-0.425086,-0.135880,-0.199716,-0.022974,0.648829,0.320250,-0.632688,6.788542,-0.0821,-0.182240,-0.164141,-0.129965,-0.057223,-0.09792,-0.120975,-0.509859
1,4.630880,0.152978,0.480835,-0.165720,-0.425086,1.384836,0.228294,5.041895,-1.602484,-1.031772,0.777376,0.524149,-0.0821,1.509276,-0.143283,-0.109344,-0.057223,-0.09792,-0.120975,-0.252385
2,-0.449545,-0.506754,0.274158,-0.165720,1.202824,-0.254462,-0.199716,-0.416141,-0.915459,2.196868,-0.917054,-0.234968,-0.0821,-0.313260,-0.163517,-0.129965,-0.057223,-0.09792,-0.120975,-0.253413
3,-0.442206,-0.424066,-0.341355,1.188042,-0.177377,-0.768535,-0.199716,-0.416141,0.075200,0.571765,-0.800220,-0.234968,-0.0821,-0.312721,-0.164966,-0.129965,-0.057223,-0.09792,-0.120975,1.994533
4,-0.436615,-0.580487,-0.762408,-0.165720,-0.425086,-0.637511,-0.199716,-0.414059,-0.545487,1.596475,1.499602,-0.234968,-0.0821,-0.313079,-0.164966,-0.129965,-0.057223,-0.09792,-0.120975,-0.641539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,1.493791,1.327758,3.745043,-0.165720,-0.425014,1.904319,0.162446,4.195504,-1.635092,-1.033457,-0.893181,2.859198,-0.0821,0.031001,0.003747,-0.129965,-0.057223,-0.09792,-0.120975,-0.638894
303,-0.449545,-0.580543,-0.759939,-0.165720,-0.387349,-0.843036,-0.199716,-0.416141,1.738015,-1.033106,0.014166,-0.234968,-0.0821,-0.313115,-0.164188,-0.129965,-0.057223,-0.09792,-0.120975,-0.651851
304,3.815256,0.122109,0.362332,-0.163790,-0.380441,-0.803680,-0.169829,-0.363583,-0.977271,-0.662671,0.801773,0.086202,-0.0821,-0.248569,-0.164966,-0.129965,-0.057223,-0.09792,0.070326,-0.481383
305,-0.449545,0.163930,0.411797,-0.165720,3.356586,-0.351171,-0.199716,-0.416141,-0.711900,-0.884083,-0.504183,-0.234968,-0.0821,-0.313260,-0.163592,-0.129965,-0.057223,-0.09792,3.329384,0.301915


### Ratings

In [ ]:
rates_ok = pd.read_csv("/content/drive/My Drive/street_psych/ratingsAveraged_images307.csv", index_col=0)
clean_rates_ok = standardize_data(rates_ok)
clean_rates_ok.columns=['Naturalness', 'Preference']
clean_rates_ok

,Naturalness,Preference
0,-0.672159,0.238289
1,-0.477989,0.388593
2,-0.192872,0.714356
3,-0.436232,-0.824860
4,-0.488812,-0.305902
...,...,...
302,-0.814087,-0.844359
303,-1.243072,-1.071693
304,-0.013219,-1.056080
305,-1.302404,-1.864314


#  Prep & OLS Regression

## Concatenate DataFrames




###Sidewalk

In [ ]:
all_sw = combine_ratings_and_features(clean_lo_sw, clean_mid_sw, clean_rates_sw)
all_sw

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled,Walkability,Preference,Imageability,Complexity,Enclosure,Transparency,Humanscale,Disorder
0,-0.697388,0.582741,-0.829933,0.965938,1.020501,-1.769262,1.537084,0.988298,-1.645022,0.580355,1.606022,-0.390942,-0.677097,-0.190748,-0.675653,0.087769,-0.335653,0.021891,-0.403206,-0.539987,1.201695,-0.026431,0.561862,-0.626202,-0.216990,-0.125748,-0.042145,-0.046379,-0.093707,-0.574748,-0.328069,-0.396165,-0.338632,-0.538060,-0.332572,0.167202,0.599738,-0.738702
1,0.142487,0.244800,1.469235,0.135219,-0.683353,1.410906,-0.049747,0.706928,0.197814,-0.011075,0.079553,1.598872,-0.161347,-0.190748,-0.367752,-0.369828,-0.374001,-0.547766,0.160267,-0.403506,-0.084766,1.054426,-0.145018,-0.597545,0.129925,-0.130470,-0.042145,-0.088317,0.659986,-0.041967,1.841204,2.017750,2.287950,2.898411,-0.820297,1.115442,-1.113798,0.084024
2,1.043314,-2.309889,-1.146498,-0.854800,0.088301,-0.794455,-0.971109,-0.106545,2.490735,-1.105760,-0.672936,-0.398976,1.465504,-0.190748,-0.212524,-0.491069,-0.403608,-0.441761,0.045671,-0.297560,-1.195210,0.658537,-0.145018,0.916010,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.599589,-0.006695,-0.038547,-0.229192,0.934713,0.837968,1.984662,-0.314148,-0.224498
3,0.496954,-0.299344,0.456589,0.652278,0.120898,1.793095,1.255978,0.477388,1.035100,-0.341145,0.804310,-0.935804,0.180557,-0.190748,-0.604152,-0.605742,-0.405800,-0.464203,0.614335,-0.339743,-0.421392,-0.169500,-0.145018,-0.110990,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.589360,1.118113,1.481325,1.521864,2.014747,0.252698,1.589562,-0.085677,-0.430180
4,-1.501323,0.031841,-0.651723,1.288547,1.839265,-0.115493,0.271976,-0.040704,-0.620530,-0.227754,1.493925,-0.897668,-1.291167,-0.167972,0.044460,-0.486964,-0.400332,-0.102259,-0.603201,-0.570708,2.104099,-0.313549,-0.145018,-0.253864,-0.090894,0.165332,-0.042145,-0.088317,-0.247871,-0.361748,-0.810130,-0.217356,-0.448073,0.934713,-1.210477,-0.860058,-1.113798,0.803909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,-0.601862,1.133674,-0.182982,0.640171,1.050453,-0.545005,-0.646316,0.035079,-0.142850,-0.416623,0.151127,-0.908833,-0.550650,-0.187266,-0.268936,-0.235177,1.139660,0.210504,-0.872190,-0.606905,1.300611,0.472351,-0.145018,1.860915,-0.198197,-0.130470,-0.042145,-0.088317,0.594731,0.018056,-0.729786,-0.664377,-0.995278,1.327453,-1.405567,0.167202,-1.799213,1.523795
548,-1.122581,0.485214,-1.280342,1.756527,1.128100,-0.687161,0.638593,0.102449,-0.688028,-0.202492,1.640049,-1.083393,-0.971368,-0.190748,-0.697320,-0.429765,-0.317020,-0.744973,-0.793493,-0.599387,1.895585,0.081361,-0.145018,-0.488606,1.682480,-0.130470,-0.042145,-0.088317,-0.246240,0.148328,-0.970817,-1.111399,-1.433042,-0.734430,-1.112932,-0.860058,-0.199913,-0.533021
549,-0.286593,0.009437,-0.624113,0.086371,0.142504,-0.198892,0.065347,1.381106,0.761069,-0.806091,-1.441450,0.498869,1.073859,0.398207,1.066128,0.569176,-0.223853,0.779206,-1.123328,-0.607091,-0.309367,3.410656,-0.145018,1.464621,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,0.269947,-1.211847,-1.290207,-0.557514,0.836528,-0.625207,1.431522,0.713973,2.140840
550,-0.766012,0.261837,-1.473116,1.839824,0.420205,-0.645071,0.151189,0.014913,-0.113812,-0.389604,1.932614,-0.922084,-0.619306,-0.167570,-0.836452,-0.281155,-0.406898,0.184241,-0.824157,-0.523930,1.667382,0.320952,-0.145018,-0.675682,-0.219113,-0.130470,-0.042145,-0.088317,-0.247871,-0.581155,-0.970817,-0.843186,-1.214160,-1.028985,-0.235027,-0.227898,-0.885327,-0.430180


###Road

In [ ]:
all_rd = combine_ratings_and_features(clean_lo_rd, clean_mid_rd, clean_rates_rd)
all_rd

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled,Walkability,Preference,Imageability,Complexity,Enclosure,Transparency,Humanscale,Disorder
0,0.082438,0.550033,0.790950,-0.198505,-0.070003,0.153939,0.129149,0.967512,0.161195,-0.091117,-0.311138,1.531860,-0.835667,-0.017142,-0.739186,-0.216989,-0.425122,2.940363,0.362426,0.526167,0.146232,-0.358936,-0.129245,-0.619744,1.939384,-0.064760,-0.042145,-0.068924,-0.240763,-0.104711,-0.328069,-0.396165,-0.338632,-0.538060,-0.332572,0.167202,0.599738,-0.738702
1,-0.788018,1.002600,0.438174,-0.185384,1.546090,-0.193184,-0.979389,-1.424131,-0.621514,-0.139558,0.864339,-0.300955,0.206693,-0.147998,-0.774902,-0.560783,-0.391109,-0.236009,-0.524059,-0.508972,0.805189,-0.042003,-0.129245,-0.719019,-0.322795,-0.166632,-0.042145,-0.068924,-0.240763,-0.286816,1.841204,2.017750,2.287950,2.898411,-0.820297,1.115442,-1.113798,0.084024
2,-0.157772,-2.151727,-0.683065,-0.138796,-0.468528,-0.278418,0.824622,1.271180,0.667325,-0.710098,-0.580077,0.206287,0.991756,-0.147998,-0.835441,-0.279497,-0.218089,-0.580116,-0.593476,-0.653270,0.228581,-0.358936,-0.129245,0.622919,-0.053923,-0.163850,-0.042145,-0.068924,-0.240763,-0.221381,-0.006695,-0.038547,-0.229192,0.934713,0.837968,1.984662,-0.314148,-0.224498
3,-0.697148,-1.906191,-0.246839,0.768918,0.729539,0.294893,1.456231,0.314191,0.335352,-0.468591,1.077610,-0.273756,-0.369115,-0.147998,0.874103,1.866932,-0.185556,-0.299536,-0.166820,-0.393211,0.329094,-0.266795,-0.129245,-0.721203,-0.299559,-0.152029,-0.042145,-0.068924,-0.034081,-0.251799,1.118113,1.481325,1.521864,2.014747,0.252698,1.589562,-0.085677,-0.430180
4,-0.830773,0.180614,0.770656,0.015852,0.949692,1.130193,-0.703302,-0.201237,-0.471402,-0.370473,0.846085,-0.985651,-0.882177,-0.147998,-0.627931,0.078295,-0.426600,-1.017245,-0.299888,-0.635736,0.983316,-0.356553,-0.129245,0.729099,-0.326778,-0.152377,-0.042145,-0.068924,-0.240763,0.555692,-0.810130,-0.217356,-0.448073,0.934713,-1.210477,-0.860058,-1.113798,0.803909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,-0.707258,1.350689,-0.401609,-0.071152,0.323305,-0.841767,-0.747521,0.160509,0.843873,-1.247143,-0.568511,-0.688446,-0.150352,-0.147998,-0.756687,-0.811140,1.186773,-0.576334,-0.738174,-0.653270,0.849797,2.378285,-0.129245,2.754196,-0.251096,-0.166632,-0.042145,-0.068924,1.233207,-0.197660,-0.729786,-0.664377,-0.995278,1.327453,-1.405567,0.167202,-1.799213,1.523795
548,-1.431420,0.434880,-1.619145,1.646230,-0.174660,-0.852298,-0.039796,-0.377581,-0.548982,0.537702,1.008263,-0.232674,-0.264446,0.155987,0.548552,-0.424047,-0.426600,0.690433,-1.112475,-0.648618,1.363411,-0.358936,-0.129245,-0.226308,-0.323459,-0.166632,-0.042145,-0.068924,-0.240763,-0.105356,-0.970817,-1.111399,-1.433042,-0.734430,-1.112932,-0.860058,-0.199913,-0.533021
549,-0.888877,0.341988,-1.034897,0.982797,0.204718,-0.842375,-0.132077,-0.333057,-0.459048,0.106841,-0.021933,0.987030,1.030121,-0.147998,-0.721328,-0.529529,-0.426600,-0.131642,-1.181106,-0.653270,0.584835,1.533924,-0.129245,-0.170012,-0.327110,-0.166632,-0.042145,-0.068924,-0.180934,-0.189672,-1.211847,-1.290207,-0.557514,0.836528,-0.625207,1.431522,0.713973,2.140840
550,-0.215875,0.249791,-1.115426,0.442784,-0.385735,-0.516379,0.047866,0.799870,0.292311,0.217916,-0.336835,1.060977,1.247690,-0.109568,0.558374,1.745498,-0.426600,2.682472,-1.183465,-0.653270,-0.003596,0.288433,-0.129245,0.067712,-0.325451,-0.166632,-0.042145,-0.068924,-0.240763,-0.258254,-0.970817,-0.843186,-1.214160,-1.028985,-0.235027,-0.227898,-0.885327,-0.430180


###Emily Talen

In [ ]:
all_et = combine_ratings_and_features(clean_lo_et, clean_mid_et, clean_rates_et)
all_et

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled,Walkability,Preference,Imageability,Complexity,Enclosure,Transparency,Humanscale,Disorder
0,1.265458,-0.851859,-0.270888,0.372972,0.158647,-0.011795,-0.293687,0.853004,-0.491502,1.402001,-1.182648,2.132680,-1.585717,-0.191013,0.364651,1.029592,-0.574475,-0.282861,1.584209,0.869271,-0.260070,0.281507,-0.168500,0.769642,-0.307147,-0.081179,-0.045038,-0.096961,-0.223434,-0.162660,-1.083078,0.327144,-0.721013,0.447036,-0.715858,-1.116452,0.781183,0.970461
1,0.091315,1.524110,-0.156125,-0.116810,1.790921,-0.035404,0.548584,1.519825,0.769589,-0.219329,-0.629600,-0.853475,-0.012611,-0.191013,-0.628944,-0.061473,-0.674496,0.456005,0.316914,-0.382630,-0.245267,-0.377845,-0.168500,3.429470,-0.087651,-0.257506,-0.045038,-0.096961,-0.223434,-0.254883,0.900115,0.459700,0.411867,1.957607,0.197174,1.425430,-1.070382,1.586034
2,-0.525075,1.727045,-0.176056,0.641278,-0.557655,-0.218428,2.263158,1.036623,-0.680173,0.497248,-0.391237,-0.476075,-1.410169,-0.091038,0.401927,-0.937853,-0.754019,-0.690603,0.538499,0.129382,1.272925,-0.530266,-0.168500,-0.763071,0.385831,-0.239085,-0.045038,-0.096961,-0.223434,1.005093,-1.477525,-0.854817,-1.051695,-1.361834,-1.014728,-1.488415,-0.427478,-0.828639
3,0.234903,-1.671054,0.576315,-0.883231,-1.051611,1.093536,2.112478,0.209921,-0.145027,-0.322996,-0.798453,-0.834788,-0.781849,-0.191013,1.794002,-0.660562,-0.744084,-0.729229,0.529481,-0.234995,0.254465,-0.057072,-0.168500,-0.866147,0.414265,-0.257506,-0.045038,-0.096961,-0.223434,1.896127,-0.091629,-0.496312,-0.506069,-0.843381,-0.442441,0.545975,0.091711,-0.558512
4,-0.778060,2.022104,0.457735,0.573101,-0.052567,0.645432,2.083170,0.981226,0.352090,-1.034856,-0.862211,-1.065663,-0.318065,-0.154715,-0.253409,1.820911,1.593102,0.009532,-0.750045,-0.380199,0.540548,-0.499765,-0.168500,0.654618,-0.400614,-0.257506,-0.045038,-0.096961,-0.223434,1.597656,-0.752251,-0.197558,0.380574,-0.815531,-0.472820,-0.160885,0.413035,0.135631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,2.535470,0.381666,-0.908047,0.016100,-0.001855,-0.976698,-0.542233,0.927218,-1.704212,2.669128,-0.070108,-0.597025,-0.870212,-0.191013,-0.653160,-1.020278,-0.598415,-0.603192,1.791928,6.863127,-0.843106,-0.445238,-0.168500,1.382717,-0.400614,-0.257506,-0.045038,1.646961,0.998227,-0.346011,0.798999,-0.364145,0.138762,-0.159967,0.016452,0.296189,0.291880,-0.760207
1115,1.910454,0.858489,-0.937488,0.081708,-0.385447,-0.725969,1.123742,0.018588,-1.507824,2.410881,0.908227,-1.088441,-1.377703,1.427248,-0.272184,-1.326267,-0.751377,-0.300534,3.413975,-0.356800,-0.354866,-0.529243,-0.168500,0.399087,-0.400614,-0.257506,-0.045038,-0.096961,-0.223434,-0.649815,-0.031218,1.103546,-0.237680,-0.125628,-0.770221,-1.483536,1.086838,0.152220
1116,1.713471,0.498388,-0.414334,-0.727826,-0.295367,-0.232528,-0.537495,1.300442,-0.921338,2.269649,1.654465,-0.629249,-0.889648,-0.191013,0.777099,-1.280959,-0.715072,-0.713779,2.602343,-0.192530,-1.100384,-0.512033,-0.168500,0.679978,-0.271558,-0.257506,-0.045038,-0.096961,-0.212060,-0.627752,0.508313,0.327144,-0.033193,0.156313,0.249540,-1.034702,1.978154,0.671602
1117,-0.153634,0.588014,-0.766334,0.880246,-0.617186,-0.680503,-0.210218,-0.090134,-0.694914,0.039262,2.390380,0.092292,-0.713610,-0.191013,-0.077320,0.353920,-0.371189,1.661664,0.456971,-0.366943,-0.422818,-0.133240,0.157199,-0.420575,-0.397684,0.726936,-0.045038,-0.096961,0.061209,-0.473040,0.112988,0.209316,-0.033193,0.293368,-0.745416,-0.073899,1.510053,-1.115314


###Img307/Omid

In [ ]:
all_ok = combine_ratings_and_features(clean_lo_ok, clean_mid_ok, clean_rates_ok)
all_ok

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled,Naturalness,Preference
0,-1.232490,-0.082796,0.119985,1.617739,-0.497826,0.820966,-0.950572,0.814664,1.185487,-1.412920,-0.449545,-0.576600,-0.115634,-0.165720,-0.425086,-0.135880,-0.199716,-0.022974,0.648829,0.320250,-0.632688,6.788542,-0.0821,-0.182240,-0.164141,-0.129965,-0.057223,-0.09792,-0.120975,-0.509859,-0.672159,0.238289
1,-0.617268,-0.427681,-1.478633,-0.497113,1.166007,-0.206832,-0.572974,-0.495379,1.508678,-1.249677,4.630880,0.152978,0.480835,-0.165720,-0.425086,1.384836,0.228294,5.041895,-1.602484,-1.031772,0.777376,0.524149,-0.0821,1.509276,-0.143283,-0.109344,-0.057223,-0.09792,-0.120975,-0.252385,-0.477989,0.388593
2,-0.113970,-0.090056,-1.625161,0.802979,-0.230390,-0.739948,-1.476310,0.405306,0.007830,-0.371388,-0.449545,-0.506754,0.274158,-0.165720,1.202824,-0.254462,-0.199716,-0.416141,-0.915459,2.196868,-0.917054,-0.234968,-0.0821,-0.313260,-0.163517,-0.129965,-0.057223,-0.09792,-0.120975,-0.253413,-0.192872,0.714356
3,1.550131,0.652147,-0.361839,-0.691105,-0.119287,-0.383548,-0.880278,0.794122,-0.997045,1.341663,-0.442206,-0.424066,-0.341355,1.188042,-0.177377,-0.768535,-0.199716,-0.416141,0.075200,0.571765,-0.800220,-0.234968,-0.0821,-0.312721,-0.164966,-0.129965,-0.057223,-0.09792,-0.120975,1.994533,-0.436232,-0.824860
4,2.226447,0.308501,1.523688,-0.232258,-1.580803,0.470686,0.352914,1.316032,0.178628,1.383540,-0.436615,-0.580487,-0.762408,-0.165720,-0.425086,-0.637511,-0.199716,-0.414059,-0.545487,1.596475,1.499602,-0.234968,-0.0821,-0.313079,-0.164966,-0.129965,-0.057223,-0.09792,-0.120975,-0.641539,-0.488812,-0.305902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,-1.081626,0.728927,-1.460687,-0.377715,0.385941,-0.457906,1.194757,0.361908,-0.709314,-0.552956,1.493791,1.327758,3.745043,-0.165720,-0.425014,1.904319,0.162446,4.195504,-1.635092,-1.033457,-0.893181,2.859198,-0.0821,0.031001,0.003747,-0.129965,-0.057223,-0.09792,-0.120975,-0.638894,-0.814087,-0.844359
303,0.676321,0.376195,0.719431,-0.969375,-0.622591,1.309998,-0.968416,0.767026,0.340145,0.297682,-0.449545,-0.580543,-0.759939,-0.165720,-0.387349,-0.843036,-0.199716,-0.416141,1.738015,-1.033106,0.014166,-0.234968,-0.0821,-0.313115,-0.164188,-0.129965,-0.057223,-0.09792,-0.120975,-0.651851,-1.243072,-1.071693
304,0.814262,0.221237,-0.319040,-0.927605,-0.916105,0.016698,-0.417879,0.924877,-0.964916,1.020624,3.815256,0.122109,0.362332,-0.163790,-0.380441,-0.803680,-0.169829,-0.363583,-0.977271,-0.662671,0.801773,0.086202,-0.0821,-0.248569,-0.164966,-0.129965,-0.057223,-0.09792,0.070326,-0.481383,-0.013219,-1.056080
305,1.503559,0.279991,-0.061190,0.192838,-1.692653,-1.070291,-2.670133,-1.034059,-0.649023,1.168845,-0.449545,0.163930,0.411797,-0.165720,3.356586,-0.351171,-0.199716,-0.416141,-0.711900,-0.884083,-0.504183,-0.234968,-0.0821,-0.313260,-0.163592,-0.129965,-0.057223,-0.09792,3.329384,0.301915,-1.302404,-1.864314


## Isolate Ratings

###Preference

In [ ]:
pref_sw = all_sw['Preference']    
pref_rd = all_rd['Preference']
pref_et = all_et['Preference']
pref_ok = all_ok['Preference']

In [ ]:
pref_sw
pref_rd
pref_et
pref_ok

0      0.238289
1      0.388593
2      0.714356
3     -0.824860
4     -0.305902
         ...   
302   -0.844359
303   -1.071693
304   -1.056080
305   -1.864314
306   -0.385358
Name: Preference, Length: 307, dtype: float64

###Walkability


In [ ]:
walk_sw = all_sw['Walkability']    
walk_rd = all_rd['Walkability']
walk_et = all_et['Walkability']
#walk_ok = all_ok['Walkability'] - no walkability included

###Imageability


In [ ]:
img_sw = all_sw['Imageability']
img_rd = all_rd['Imageability']
img_et = all_et['Imageability']
#img_ok = all_ok['Imageability'] - no img included

###Complexity

In [ ]:
comp_sw = all_sw['Complexity']    
comp_rd = all_rd['Complexity']
comp_et = all_et['Complexity']
#comp_ok = all_ok['Complexity'] - no comp included

###Enclosure

In [ ]:
en_sw = all_sw['Enclosure']    
en_rd = all_rd['Enclosure']
en_et = all_et['Enclosure']
#en_ok = all_ok['Enclosure'] - no en included

###Transparency

In [ ]:
tr_sw = all_sw['Transparency']    
tr_rd = all_rd['Transparency']
tr_et = all_et['Transparency']
#tr_ok = all_ok['Transparency'] - no tr included

###Humanscale

In [ ]:
hs_sw = all_sw['Humanscale']    
hs_rd = all_rd['Humanscale']
hs_et = all_et['Humanscale']
#hs_ok = all_ok['Humanscale'] - no hs included

###Disorder


In [ ]:
dis_sw = all_sw['Disorder']    
dis_rd = all_rd['Disorder']
dis_et = all_et['Disorder']
#dis_ok = all_ok['Disorder'] - no dis included

## Concatenate Ratings

***Note that all features except preference are sans OK (OK data only has naturalness and pref)***

---
*function below* 


In [ ]:
def concat_all(*args):
  l = []
  for a in args:
    l.append(a)
  all = pd.concat(l, axis=0)
  all = pd.DataFrame(all)
  return all

###All Preference

In [ ]:
all_pref = concat_all(pref_sw, pref_rd, pref_et, pref_ok)
all_pref

,Preference
0,-0.396165
1,2.017750
2,-0.038547
3,1.481325
4,-0.217356
...,...
302,-0.844359
303,-1.071693
304,-1.056080
305,-1.864314


### All Walkability

In [ ]:
all_walk = concat_all(walk_sw, walk_rd, walk_et)
all_walk

,Walkability
0,-0.328069
1,1.841204
2,-0.006695
3,1.118113
4,-0.810130
...,...
1114,0.798999
1115,-0.031218
1116,0.508313
1117,0.112988


### All Imageability


In [ ]:
all_img = concat_all(img_sw, img_rd, img_et)
all_img

,Imageability
0,-0.338632
1,2.287950
2,-0.229192
3,1.521864
4,-0.448073
...,...
1114,0.138762
1115,-0.237680
1116,-0.033193
1117,-0.033193


###All Complexity

In [ ]:
all_comp = concat_all(comp_sw, comp_rd, comp_et)
all_comp

,Complexity
0,-0.538060
1,2.898411
2,0.934713
3,2.014747
4,0.934713
...,...
1114,-0.159967
1115,-0.125628
1116,0.156313
1117,0.293368


###All Enclosure

In [ ]:
all_en = concat_all(en_sw, en_rd, en_et)
all_en

,Enclosure
0,-0.332572
1,-0.820297
2,0.837968
3,0.252698
4,-1.210477
...,...
1114,0.016452
1115,-0.770221
1116,0.249540
1117,-0.745416


###All Transparency

In [ ]:
all_tr = concat_all(tr_sw, tr_rd, tr_et)
all_tr

,Transparency
0,0.167202
1,1.115442
2,1.984662
3,1.589562
4,-0.860058
...,...
1114,0.296189
1115,-1.483536
1116,-1.034702
1117,-0.073899


###All Humanscale

In [ ]:
all_hs = concat_all(hs_sw, hs_rd, hs_et)
all_hs

,Humanscale
0,0.599738
1,-1.113798
2,-0.314148
3,-0.085677
4,-1.113798
...,...
1114,0.291880
1115,1.086838
1116,1.978154
1117,1.510053


###All Disorder


In [ ]:
all_dis = concat_all(dis_sw, dis_rd, dis_et)
all_dis

,Disorder
0,-0.738702
1,0.084024
2,-0.224498
3,-0.430180
4,0.803909
...,...
1114,-0.760207
1115,0.152220
1116,0.671602
1117,-1.115314


## Concatenate Features

### All Low

In [ ]:
all_lo = concat_all(clean_lo_sw, clean_lo_rd, clean_lo_et, clean_lo_ok)
all_lo

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED
0,-0.697388,0.582741,-0.829933,0.965938,1.020501,-1.769262,1.537084,0.988298,-1.645022,0.580355
1,0.142487,0.244800,1.469235,0.135219,-0.683353,1.410906,-0.049747,0.706928,0.197814,-0.011075
2,1.043314,-2.309889,-1.146498,-0.854800,0.088301,-0.794455,-0.971109,-0.106545,2.490735,-1.105760
3,0.496954,-0.299344,0.456589,0.652278,0.120898,1.793095,1.255978,0.477388,1.035100,-0.341145
4,-1.501323,0.031841,-0.651723,1.288547,1.839265,-0.115493,0.271976,-0.040704,-0.620530,-0.227754
...,...,...,...,...,...,...,...,...,...,...
302,-1.081626,0.728927,-1.460687,-0.377715,0.385941,-0.457906,1.194757,0.361908,-0.709314,-0.552956
303,0.676321,0.376195,0.719431,-0.969375,-0.622591,1.309998,-0.968416,0.767026,0.340145,0.297682
304,0.814262,0.221237,-0.319040,-0.927605,-0.916105,0.016698,-0.417879,0.924877,-0.964916,1.020624
305,1.503559,0.279991,-0.061190,0.192838,-1.692653,-1.070291,-2.670133,-1.034059,-0.649023,1.168845


### All Mid

In [ ]:
all_mid = concat_all(clean_mid_sw, clean_mid_rd, clean_mid_et, clean_mid_ok)
all_mid

,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,1.606022,-0.390942,-0.677097,-0.190748,-0.675653,0.087769,-0.335653,0.021891,-0.403206,-0.539987,1.201695,-0.026431,0.561862,-0.626202,-0.216990,-0.125748,-0.042145,-0.046379,-0.093707,-0.574748
1,0.079553,1.598872,-0.161347,-0.190748,-0.367752,-0.369828,-0.374001,-0.547766,0.160267,-0.403506,-0.084766,1.054426,-0.145018,-0.597545,0.129925,-0.130470,-0.042145,-0.088317,0.659986,-0.041967
2,-0.672936,-0.398976,1.465504,-0.190748,-0.212524,-0.491069,-0.403608,-0.441761,0.045671,-0.297560,-1.195210,0.658537,-0.145018,0.916010,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.599589
3,0.804310,-0.935804,0.180557,-0.190748,-0.604152,-0.605742,-0.405800,-0.464203,0.614335,-0.339743,-0.421392,-0.169500,-0.145018,-0.110990,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.589360
4,1.493925,-0.897668,-1.291167,-0.167972,0.044460,-0.486964,-0.400332,-0.102259,-0.603201,-0.570708,2.104099,-0.313549,-0.145018,-0.253864,-0.090894,0.165332,-0.042145,-0.088317,-0.247871,-0.361748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,1.493791,1.327758,3.745043,-0.165720,-0.425014,1.904319,0.162446,4.195504,-1.635092,-1.033457,-0.893181,2.859198,-0.082100,0.031001,0.003747,-0.129965,-0.057223,-0.097920,-0.120975,-0.638894
303,-0.449545,-0.580543,-0.759939,-0.165720,-0.387349,-0.843036,-0.199716,-0.416141,1.738015,-1.033106,0.014166,-0.234968,-0.082100,-0.313115,-0.164188,-0.129965,-0.057223,-0.097920,-0.120975,-0.651851
304,3.815256,0.122109,0.362332,-0.163790,-0.380441,-0.803680,-0.169829,-0.363583,-0.977271,-0.662671,0.801773,0.086202,-0.082100,-0.248569,-0.164966,-0.129965,-0.057223,-0.097920,0.070326,-0.481383
305,-0.449545,0.163930,0.411797,-0.165720,3.356586,-0.351171,-0.199716,-0.416141,-0.711900,-0.884083,-0.504183,-0.234968,-0.082100,-0.313260,-0.163592,-0.129965,-0.057223,-0.097920,3.329384,0.301915


### All Low (sans OK)

In [ ]:
all_lo_no_ok = concat_all(clean_lo_sw, clean_lo_rd, clean_lo_et)
all_lo_no_ok

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED
0,-0.697388,0.582741,-0.829933,0.965938,1.020501,-1.769262,1.537084,0.988298,-1.645022,0.580355
1,0.142487,0.244800,1.469235,0.135219,-0.683353,1.410906,-0.049747,0.706928,0.197814,-0.011075
2,1.043314,-2.309889,-1.146498,-0.854800,0.088301,-0.794455,-0.971109,-0.106545,2.490735,-1.105760
3,0.496954,-0.299344,0.456589,0.652278,0.120898,1.793095,1.255978,0.477388,1.035100,-0.341145
4,-1.501323,0.031841,-0.651723,1.288547,1.839265,-0.115493,0.271976,-0.040704,-0.620530,-0.227754
...,...,...,...,...,...,...,...,...,...,...
1114,2.535470,0.381666,-0.908047,0.016100,-0.001855,-0.976698,-0.542233,0.927218,-1.704212,2.669128
1115,1.910454,0.858489,-0.937488,0.081708,-0.385447,-0.725969,1.123742,0.018588,-1.507824,2.410881
1116,1.713471,0.498388,-0.414334,-0.727826,-0.295367,-0.232528,-0.537495,1.300442,-0.921338,2.269649
1117,-0.153634,0.588014,-0.766334,0.880246,-0.617186,-0.680503,-0.210218,-0.090134,-0.694914,0.039262


### All Mid (sans OK)

In [ ]:
all_mid_no_ok = concat_all(clean_mid_sw, clean_mid_rd, clean_mid_et)
all_mid_no_ok

,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,1.606022,-0.390942,-0.677097,-0.190748,-0.675653,0.087769,-0.335653,0.021891,-0.403206,-0.539987,1.201695,-0.026431,0.561862,-0.626202,-0.216990,-0.125748,-0.042145,-0.046379,-0.093707,-0.574748
1,0.079553,1.598872,-0.161347,-0.190748,-0.367752,-0.369828,-0.374001,-0.547766,0.160267,-0.403506,-0.084766,1.054426,-0.145018,-0.597545,0.129925,-0.130470,-0.042145,-0.088317,0.659986,-0.041967
2,-0.672936,-0.398976,1.465504,-0.190748,-0.212524,-0.491069,-0.403608,-0.441761,0.045671,-0.297560,-1.195210,0.658537,-0.145018,0.916010,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.599589
3,0.804310,-0.935804,0.180557,-0.190748,-0.604152,-0.605742,-0.405800,-0.464203,0.614335,-0.339743,-0.421392,-0.169500,-0.145018,-0.110990,-0.219718,-0.130470,-0.042145,-0.088317,-0.247871,-0.589360
4,1.493925,-0.897668,-1.291167,-0.167972,0.044460,-0.486964,-0.400332,-0.102259,-0.603201,-0.570708,2.104099,-0.313549,-0.145018,-0.253864,-0.090894,0.165332,-0.042145,-0.088317,-0.247871,-0.361748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,-0.070108,-0.597025,-0.870212,-0.191013,-0.653160,-1.020278,-0.598415,-0.603192,1.791928,6.863127,-0.843106,-0.445238,-0.168500,1.382717,-0.400614,-0.257506,-0.045038,1.646961,0.998227,-0.346011
1115,0.908227,-1.088441,-1.377703,1.427248,-0.272184,-1.326267,-0.751377,-0.300534,3.413975,-0.356800,-0.354866,-0.529243,-0.168500,0.399087,-0.400614,-0.257506,-0.045038,-0.096961,-0.223434,-0.649815
1116,1.654465,-0.629249,-0.889648,-0.191013,0.777099,-1.280959,-0.715072,-0.713779,2.602343,-0.192530,-1.100384,-0.512033,-0.168500,0.679978,-0.271558,-0.257506,-0.045038,-0.096961,-0.212060,-0.627752
1117,2.390380,0.092292,-0.713610,-0.191013,-0.077320,0.353920,-0.371189,1.661664,0.456971,-0.366943,-0.422818,-0.133240,0.157199,-0.420575,-0.397684,0.726936,-0.045038,-0.096961,0.061209,-0.473040


### All Features

In [ ]:
all_feat = concat_all(all_lo, all_mid)
all_feat

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,-0.697388,0.582741,-0.829933,0.965938,1.020501,-1.769262,1.537084,0.988298,-1.645022,0.580355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.142487,0.244800,1.469235,0.135219,-0.683353,1.410906,-0.049747,0.706928,0.197814,-0.011075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.043314,-2.309889,-1.146498,-0.854800,0.088301,-0.794455,-0.971109,-0.106545,2.490735,-1.105760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.496954,-0.299344,0.456589,0.652278,0.120898,1.793095,1.255978,0.477388,1.035100,-0.341145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.501323,0.031841,-0.651723,1.288547,1.839265,-0.115493,0.271976,-0.040704,-0.620530,-0.227754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.493791,1.327758,3.745043,-0.16572,-0.425014,1.904319,0.162446,4.195504,-1.635092,-1.033457,-0.893181,2.859198,-0.0821,0.031001,0.003747,-0.129965,-0.057223,-0.09792,-0.120975,-0.638894
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.449545,-0.580543,-0.759939,-0.16572,-0.387349,-0.843036,-0.199716,-0.416141,1.738015,-1.033106,0.014166,-0.234968,-0.0821,-0.313115,-0.164188,-0.129965,-0.057223,-0.09792,-0.120975,-0.651851
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.815256,0.122109,0.362332,-0.16379,-0.380441,-0.803680,-0.169829,-0.363583,-0.977271,-0.662671,0.801773,0.086202,-0.0821,-0.248569,-0.164966,-0.129965,-0.057223,-0.09792,0.070326,-0.481383
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.449545,0.163930,0.411797,-0.16572,3.356586,-0.351171,-0.199716,-0.416141,-0.711900,-0.884083,-0.504183,-0.234968,-0.0821,-0.313260,-0.163592,-0.129965,-0.057223,-0.09792,3.329384,0.301915


### All Features (sans OK)

In [ ]:
all_feat_no_ok = concat_all(all_lo_no_ok, all_mid_no_ok)
all_feat

,ED,Hue,Sat,Lum,sdHue,sdSat,sdBright,Entropy,StraightED,NonStraightED,Road,Sidewalk,Building,Wall,Fence,Pole,Traffic Light,Traffic Sign,Vegetation,Terrain,Sky,Person,Rider,Car,Truck,Bus,Train,Motorcycle,Bicycle,Unlabeled
0,-0.697388,0.582741,-0.829933,0.965938,1.020501,-1.769262,1.537084,0.988298,-1.645022,0.580355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.142487,0.244800,1.469235,0.135219,-0.683353,1.410906,-0.049747,0.706928,0.197814,-0.011075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.043314,-2.309889,-1.146498,-0.854800,0.088301,-0.794455,-0.971109,-0.106545,2.490735,-1.105760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.496954,-0.299344,0.456589,0.652278,0.120898,1.793095,1.255978,0.477388,1.035100,-0.341145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.501323,0.031841,-0.651723,1.288547,1.839265,-0.115493,0.271976,-0.040704,-0.620530,-0.227754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.493791,1.327758,3.745043,-0.16572,-0.425014,1.904319,0.162446,4.195504,-1.635092,-1.033457,-0.893181,2.859198,-0.0821,0.031001,0.003747,-0.129965,-0.057223,-0.09792,-0.120975,-0.638894
303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.449545,-0.580543,-0.759939,-0.16572,-0.387349,-0.843036,-0.199716,-0.416141,1.738015,-1.033106,0.014166,-0.234968,-0.0821,-0.313115,-0.164188,-0.129965,-0.057223,-0.09792,-0.120975,-0.651851
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.815256,0.122109,0.362332,-0.16379,-0.380441,-0.803680,-0.169829,-0.363583,-0.977271,-0.662671,0.801773,0.086202,-0.0821,-0.248569,-0.164966,-0.129965,-0.057223,-0.09792,0.070326,-0.481383
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.449545,0.163930,0.411797,-0.16572,3.356586,-0.351171,-0.199716,-0.416141,-0.711900,-0.884083,-0.504183,-0.234968,-0.0821,-0.313260,-0.163592,-0.129965,-0.057223,-0.09792,3.329384,0.301915


##OLS Regs

### Low Level Features & Ratings

#### All of Preference & All of Low-Level Features

In [ ]:
run_reg(all_pref, all_lo)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Preference   R-squared (uncentered):                   0.193
Model:                            OLS   Adj. R-squared (uncentered):              0.190
Method:                 Least Squares   F-statistic:                              60.46
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   3.19e-110
Time:                        17:10:38   Log-Likelihood:                         -3317.9
No. Observations:                2530   AIC:                                      6656.
Df Residuals:                    2520   BIC:                                      6714.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.1155      0.040      2.913      0.004       0.038       0.193
Hue               0.0360      0.019      1.874      0.061      -0.002       0.074
Sat               0.1239      0.027      4.555      0.000       0.071       0.177
Lum              -0.1641      0.021     -7.794      0.000      -0.205      -0.123
sdHue             0.0358      0.019      1.840      0.066      -0.002       0.074
sdSat            -0.0078      0.027     -0.285      0.776      -0.062       0.046
sdBright          0.0389      0.022      1.764      0.078      -0.004       0.082
Entropy          -0.0541      0.023     -2.379      0.017      -0.099      -0.010
StraightED        0.2884      0.040      7.226      0.000       0.210       0.367
NonStraightED     0.2225      0.052      4.277      0.000       0.120       0.324
==============================================================================
Omnibus:                       19.403   Durbin-Watson:                   1.490
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.263
Skew:                           0.196   Prob(JB):                     6.56e-05
Kurtosis:                       2.831   Cond. No.                         6.52
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Walkability & All Low-Level Features (sans OK)

In [ ]:
run_reg(all_walk, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            Walkability   R-squared (uncentered):                   0.242
Model:                            OLS   Adj. R-squared (uncentered):              0.238
Method:                 Least Squares   F-statistic:                              70.52
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   2.61e-125
Time:                        17:10:38   Log-Likelihood:                         -2846.9
No. Observations:                2223   AIC:                                      5714.
Df Residuals:                    2213   BIC:                                      5771.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.1173      0.040      2.937      0.003       0.039       0.196
Hue               0.0842      0.020      4.255      0.000       0.045       0.123
Sat               0.0241      0.029      0.831      0.406      -0.033       0.081
Lum              -0.1933      0.023     -8.542      0.000      -0.238      -0.149
sdHue             0.0486      0.020      2.437      0.015       0.009       0.088
sdSat             0.0774      0.029      2.650      0.008       0.020       0.135
sdBright         -0.0097      0.024     -0.410      0.682      -0.056       0.037
Entropy          -0.0400      0.024     -1.640      0.101      -0.088       0.008
StraightED        0.2974      0.042      7.016      0.000       0.214       0.381
NonStraightED     0.2508      0.053      4.750      0.000       0.147       0.354
==============================================================================
Omnibus:                       50.001   Durbin-Watson:                   1.593
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.106
Skew:                           0.287   Prob(JB):                     2.65e-10
Kurtosis:                       2.618   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Imageability & All Low-Level Features (sans OK)

In [ ]:
run_reg(all_img, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Imageability   R-squared (uncentered):                   0.281
Model:                            OLS   Adj. R-squared (uncentered):              0.277
Method:                 Least Squares   F-statistic:                              86.34
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   1.98e-150
Time:                        17:10:38   Log-Likelihood:                         -2788.2
No. Observations:                2223   AIC:                                      5596.
Df Residuals:                    2213   BIC:                                      5653.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.0531      0.039      1.364      0.173      -0.023       0.129
Hue               0.0429      0.019      2.228      0.026       0.005       0.081
Sat               0.0384      0.028      1.363      0.173      -0.017       0.094
Lum              -0.2810      0.022    -12.750      0.000      -0.324      -0.238
sdHue             0.0631      0.019      3.246      0.001       0.025       0.101
sdSat             0.0383      0.028      1.347      0.178      -0.017       0.094
sdBright          0.0124      0.023      0.538      0.590      -0.033       0.057
Entropy          -0.0287      0.024     -1.208      0.227      -0.075       0.018
StraightED        0.3987      0.041      9.658      0.000       0.318       0.480
NonStraightED     0.2105      0.051      4.093      0.000       0.110       0.311
==============================================================================
Omnibus:                       41.614   Durbin-Watson:                   1.604
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.532
Skew:                           0.323   Prob(JB):                     5.81e-10
Kurtosis:                       2.794   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Complexity & All Low-Level Features (sans OK)

In [ ]:
run_reg(all_comp, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Complexity   R-squared (uncentered):                   0.208
Model:                            OLS   Adj. R-squared (uncentered):              0.205
Method:                 Least Squares   F-statistic:                              58.18
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   8.15e-105
Time:                        17:10:38   Log-Likelihood:                         -2894.9
No. Observations:                2223   AIC:                                      5810.
Df Residuals:                    2213   BIC:                                      5867.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED               -0.0707      0.041     -1.732      0.083      -0.151       0.009
Hue              -0.0064      0.020     -0.314      0.753      -0.046       0.033
Sat              -0.0613      0.030     -2.073      0.038      -0.119      -0.003
Lum              -0.1580      0.023     -6.834      0.000      -0.203      -0.113
sdHue             0.1658      0.020      8.131      0.000       0.126       0.206
sdSat             0.0820      0.030      2.749      0.006       0.024       0.141
sdBright          0.0216      0.024      0.896      0.370      -0.026       0.069
Entropy           0.0178      0.025      0.716      0.474      -0.031       0.067
StraightED        0.5186      0.043     11.973      0.000       0.434       0.604
NonStraightED     0.2942      0.054      5.452      0.000       0.188       0.400
==============================================================================
Omnibus:                       29.764   Durbin-Watson:                   1.536
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.279
Skew:                           0.273   Prob(JB):                     2.66e-07
Kurtosis:                       2.831   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

####All of Enclosure & Low-Level Features (sans OK)

In [ ]:
run_reg(all_en, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Enclosure   R-squared (uncentered):                   0.452
Model:                            OLS   Adj. R-squared (uncentered):              0.449
Method:                 Least Squares   F-statistic:                              182.4
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   3.60e-280
Time:                        17:10:39   Log-Likelihood:                         -2486.2
No. Observations:                2223   AIC:                                      4992.
Df Residuals:                    2213   BIC:                                      5049.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.0934      0.034      2.749      0.006       0.027       0.160
Hue               0.0906      0.017      5.383      0.000       0.058       0.124
Sat              -0.0581      0.025     -2.361      0.018      -0.106      -0.010
Lum              -0.4815      0.019    -25.026      0.000      -0.519      -0.444
sdHue            -0.0175      0.017     -1.030      0.303      -0.051       0.016
sdSat             0.0346      0.025      1.394      0.163      -0.014       0.083
sdBright          0.0085      0.020      0.422      0.673      -0.031       0.048
Entropy          -0.0209      0.021     -1.010      0.313      -0.062       0.020
StraightED        0.3155      0.036      8.754      0.000       0.245       0.386
NonStraightED     0.0446      0.045      0.993      0.321      -0.043       0.133
==============================================================================
Omnibus:                       23.093   Durbin-Watson:                   1.721
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.548
Skew:                           0.250   Prob(JB):                     7.70e-06
Kurtosis:                       3.063   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

####All of Transparency & All of Low-Level Features (sans OK)

In [ ]:
run_reg(all_tr, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Transparency   R-squared (uncentered):                   0.164
Model:                            OLS   Adj. R-squared (uncentered):              0.160
Method:                 Least Squares   F-statistic:                              43.41
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    3.99e-79
Time:                        17:10:39   Log-Likelihood:                         -2955.2
No. Observations:                2223   AIC:                                      5930.
Df Residuals:                    2213   BIC:                                      5988.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.1223      0.042      2.914      0.004       0.040       0.205
Hue              -0.0358      0.021     -1.722      0.085      -0.077       0.005
Sat              -0.0844      0.030     -2.776      0.006      -0.144      -0.025
Lum              -0.0869      0.024     -3.657      0.000      -0.133      -0.040
sdHue             0.0790      0.021      3.772      0.000       0.038       0.120
sdSat             0.0834      0.031      2.720      0.007       0.023       0.144
sdBright         -0.0172      0.025     -0.695      0.487      -0.066       0.031
Entropy           0.0466      0.026      1.817      0.069      -0.004       0.097
StraightED        0.1726      0.045      3.879      0.000       0.085       0.260
NonStraightED    -0.2752      0.055     -4.964      0.000      -0.384      -0.166
==============================================================================
Omnibus:                       68.634   Durbin-Watson:                   1.694
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.185
Skew:                           0.313   Prob(JB):                     6.30e-13
Kurtosis:                       2.537   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Humanscale & All of Low-Level Features (sans OK)

In [ ]:
run_reg(all_hs, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Humanscale   R-squared (uncentered):                   0.104
Model:                            OLS   Adj. R-squared (uncentered):              0.100
Method:                 Least Squares   F-statistic:                              25.73
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.06e-46
Time:                        17:10:39   Log-Likelihood:                         -3032.0
No. Observations:                2223   AIC:                                      6084.
Df Residuals:                    2213   BIC:                                      6141.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.2501      0.043      5.760      0.000       0.165       0.335
Hue              -0.0115      0.022     -0.536      0.592      -0.054       0.031
Sat               0.0683      0.031      2.170      0.030       0.007       0.130
Lum               0.0718      0.025      2.917      0.004       0.024       0.120
sdHue            -0.0336      0.022     -1.550      0.121      -0.076       0.009
sdSat            -0.0098      0.032     -0.309      0.757      -0.072       0.052
sdBright          0.1355      0.026      5.286      0.000       0.085       0.186
Entropy          -0.0276      0.027     -1.039      0.299      -0.080       0.024
StraightED       -0.2317      0.046     -5.030      0.000      -0.322      -0.141
NonStraightED    -0.0439      0.057     -0.765      0.445      -0.156       0.069
==============================================================================
Omnibus:                       48.248   Durbin-Watson:                   1.532
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.526
Skew:                           0.281   Prob(JB):                     5.83e-10
Kurtosis:                       2.622   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Disorder & All of Low-Level Features (sans OK)

In [ ]:
run_reg(all_dis, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               Disorder   R-squared (uncentered):                   0.066
Model:                            OLS   Adj. R-squared (uncentered):              0.061
Method:                 Least Squares   F-statistic:                              15.54
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    2.99e-27
Time:                        17:10:39   Log-Likelihood:                         -3078.9
No. Observations:                2223   AIC:                                      6178.
Df Residuals:                    2213   BIC:                                      6235.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED               -0.1530      0.044     -3.449      0.001      -0.240      -0.066
Hue              -0.1015      0.022     -4.620      0.000      -0.145      -0.058
Sat              -0.1589      0.032     -4.946      0.000      -0.222      -0.096
Lum              -0.0890      0.025     -3.543      0.000      -0.138      -0.040
sdHue             0.0709      0.022      3.200      0.001       0.027       0.114
sdSat             0.0258      0.032      0.794      0.427      -0.038       0.089
sdBright          0.1126      0.026      4.299      0.000       0.061       0.164
Entropy          -0.0128      0.027     -0.471      0.637      -0.066       0.040
StraightED        0.1086      0.047      2.308      0.021       0.016       0.201
NonStraightED     0.0660      0.059      1.126      0.260      -0.049       0.181
==============================================================================
Omnibus:                      100.469   Durbin-Watson:                   1.697
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.619
Skew:                           0.310   Prob(JB):                     2.07e-15
Kurtosis:                       2.413   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Mid-Level Features & Ratings

#### All of Preference & All of Mid-Level Features

In [ ]:
run_reg(all_pref, all_mid)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Preference   R-squared (uncentered):                   0.239
Model:                            OLS   Adj. R-squared (uncentered):              0.233
Method:                 Least Squares   F-statistic:                              39.41
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   7.91e-133
Time:                        17:10:39   Log-Likelihood:                         -3244.4
No. Observations:                2530   AIC:                                      6529.
Df Residuals:                    2510   BIC:                                      6646.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road              0.1378      0.025      5.510      0.000       0.089       0.187
Sidewalk         -0.0053      0.022     -0.236      0.813      -0.049       0.039
Building          0.2250      0.040      5.629      0.000       0.147       0.303
Wall             -0.0624      0.022     -2.797      0.005      -0.106      -0.019
Fence            -0.0036      0.019     -0.190      0.849      -0.041       0.034
Pole             -0.0242      0.020     -1.184      0.237      -0.064       0.016
Traffic Light     0.1044      0.019      5.483      0.000       0.067       0.142
Traffic Sign      0.0044      0.019      0.236      0.814      -0.032       0.041
Vegetation        0.3854      0.039     10.005      0.000       0.310       0.461
Terrain           0.0576      0.022      2.656      0.008       0.015       0.100
Sky              -0.1802      0.036     -5.006      0.000      -0.251      -0.110
Person            0.0645      0.019      3.479      0.001       0.028       0.101
Rider             0.0265      0.018      1.433      0.152      -0.010       0.063
Car              -0.0226      0.021     -1.053      0.292      -0.065       0.019
Truck            -0.0269      0.018     -1.496      0.135      -0.062       0.008
Bus              -0.0115      0.018     -0.643      0.520      -0.046       0.023
Train             0.0358      0.018      2.035      0.042       0.001       0.070
Motorcycle        0.0028      0.018      0.157      0.875      -0.032       0.037
Bicycle           0.0069      0.018      0.378      0.705      -0.029       0.043
Unlabeled         0.0355      0.023      1.564      0.118      -0.009       0.080
==============================================================================
Omnibus:                       18.223   Durbin-Watson:                   1.558
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               18.546
Skew:                           0.209   Prob(JB):                     9.39e-05
Kurtosis:                       2.962   Cond. No.                         5.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Walkability & All Mid-Level Features (sans OK)

In [ ]:
run_reg(all_walk, all_mid_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            Walkability   R-squared (uncentered):                   0.338
Model:                            OLS   Adj. R-squared (uncentered):              0.332
Method:                 Least Squares   F-statistic:                              56.23
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   1.97e-180
Time:                        17:10:39   Log-Likelihood:                         -2695.9
No. Observations:                2223   AIC:                                      5432.
Df Residuals:                    2203   BIC:                                      5546.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road             -0.0901      0.032     -2.843      0.005      -0.152      -0.028
Sidewalk         -0.1049      0.026     -4.012      0.000      -0.156      -0.054
Building         -0.0867      0.052     -1.655      0.098      -0.189       0.016
Wall             -0.1006      0.023     -4.325      0.000      -0.146      -0.055
Fence            -0.1089      0.020     -5.507      0.000      -0.148      -0.070
Pole             -0.0013      0.021     -0.061      0.951      -0.042       0.039
Traffic Light     0.0976      0.019      5.058      0.000       0.060       0.136
Traffic Sign     -0.0120      0.019     -0.640      0.522      -0.049       0.025
Vegetation        0.1874      0.047      3.966      0.000       0.095       0.280
Terrain          -0.0519      0.023     -2.236      0.025      -0.097      -0.006
Sky              -0.4740      0.047    -10.073      0.000      -0.566      -0.382
Person            0.1013      0.019      5.451      0.000       0.065       0.138
Rider             0.0315      0.019      1.687      0.092      -0.005       0.068
Car              -0.1380      0.026     -5.329      0.000      -0.189      -0.087
Truck            -0.0628      0.018     -3.469      0.001      -0.098      -0.027
Bus              -0.0282      0.018     -1.576      0.115      -0.063       0.007
Train             0.0323      0.018      1.838      0.066      -0.002       0.067
Motorcycle       -0.0059      0.017     -0.337      0.736      -0.040       0.028
Bicycle           0.0143      0.019      0.768      0.443      -0.022       0.051
Unlabeled        -0.1127      0.027     -4.164      0.000      -0.166      -0.060
==============================================================================
Omnibus:                       27.119   Durbin-Watson:                   1.773
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.935
Skew:                           0.275   Prob(JB):                     8.59e-07
Kurtosis:                       2.987   Cond. No.                         7.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Imageability & All Mid-Level Features (sans OK)

In [ ]:
run_reg(all_img, all_mid_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Imageability   R-squared (uncentered):                   0.327
Model:                            OLS   Adj. R-squared (uncentered):              0.321
Method:                 Least Squares   F-statistic:                              53.57
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   7.72e-173
Time:                        17:10:39   Log-Likelihood:                         -2713.8
No. Observations:                2223   AIC:                                      5468.
Df Residuals:                    2203   BIC:                                      5582.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road             -0.0694      0.032     -2.172      0.030      -0.132      -0.007
Sidewalk         -0.2066      0.026     -7.836      0.000      -0.258      -0.155
Building          0.0806      0.053      1.526      0.127      -0.023       0.184
Wall             -0.0796      0.023     -3.394      0.001      -0.126      -0.034
Fence            -0.0538      0.020     -2.702      0.007      -0.093      -0.015
Pole             -0.0319      0.021     -1.532      0.126      -0.073       0.009
Traffic Light     0.0921      0.019      4.734      0.000       0.054       0.130
Traffic Sign      0.0031      0.019      0.162      0.871      -0.034       0.040
Vegetation        0.1223      0.048      2.568      0.010       0.029       0.216
Terrain          -0.1052      0.023     -4.498      0.000      -0.151      -0.059
Sky              -0.4586      0.047     -9.668      0.000      -0.552      -0.366
Person            0.0785      0.019      4.191      0.000       0.042       0.115
Rider             0.0262      0.019      1.392      0.164      -0.011       0.063
Car              -0.0969      0.026     -3.710      0.000      -0.148      -0.046
Truck            -0.0203      0.018     -1.113      0.266      -0.056       0.015
Bus               0.0023      0.018      0.126      0.900      -0.033       0.038
Train             0.0287      0.018      1.619      0.106      -0.006       0.063
Motorcycle       -0.0350      0.018     -1.998      0.046      -0.069      -0.001
Bicycle           0.0160      0.019      0.853      0.394      -0.021       0.053
Unlabeled         0.0056      0.027      0.203      0.839      -0.048       0.059
==============================================================================
Omnibus:                       23.263   Durbin-Watson:                   1.736
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.722
Skew:                           0.251   Prob(JB):                     7.06e-06
Kurtosis:                       3.068   Cond. No.                         7.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Complexity & All Mid-Level Features (sans OK)

In [ ]:
run_reg(all_comp, all_mid_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Complexity   R-squared (uncentered):                   0.310
Model:                            OLS   Adj. R-squared (uncentered):              0.304
Method:                 Least Squares   F-statistic:                              49.59
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   2.84e-161
Time:                        17:10:39   Log-Likelihood:                         -2741.1
No. Observations:                2223   AIC:                                      5522.
Df Residuals:                    2203   BIC:                                      5636.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road              0.0889      0.032      2.749      0.006       0.025       0.152
Sidewalk         -0.0873      0.027     -3.269      0.001      -0.140      -0.035
Building          0.3499      0.053      6.545      0.000       0.245       0.455
Wall             -0.0866      0.024     -3.646      0.000      -0.133      -0.040
Fence            -0.0281      0.020     -1.391      0.164      -0.068       0.011
Pole              0.0077      0.021      0.366      0.714      -0.034       0.049
Traffic Light     0.1816      0.020      9.216      0.000       0.143       0.220
Traffic Sign      0.0849      0.019      4.452      0.000       0.048       0.122
Vegetation        0.3372      0.048      6.993      0.000       0.243       0.432
Terrain          -0.0915      0.024     -3.867      0.000      -0.138      -0.045
Sky              -0.0278      0.048     -0.579      0.562      -0.122       0.066
Person            0.2175      0.019     11.464      0.000       0.180       0.255
Rider             0.0327      0.019      1.719      0.086      -0.005       0.070
Car               0.1681      0.026      6.360      0.000       0.116       0.220
Truck             0.0381      0.018      2.060      0.040       0.002       0.074
Bus               0.0677      0.018      3.714      0.000       0.032       0.104
Train             0.0039      0.018      0.218      0.827      -0.031       0.039
Motorcycle       -0.0063      0.018     -0.354      0.723      -0.041       0.029
Bicycle           0.0388      0.019      2.045      0.041       0.002       0.076
Unlabeled         0.1174      0.028      4.250      0.000       0.063       0.172
==============================================================================
Omnibus:                       17.015   Durbin-Watson:                   1.726
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.333
Skew:                           0.215   Prob(JB):                     0.000172
Kurtosis:                       2.956   Cond. No.                         7.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

####All of Enclosure & Low-Level Features (sans OK)

In [ ]:
run_reg(all_en, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Enclosure   R-squared (uncentered):                   0.452
Model:                            OLS   Adj. R-squared (uncentered):              0.449
Method:                 Least Squares   F-statistic:                              182.4
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   3.60e-280
Time:                        17:10:39   Log-Likelihood:                         -2486.2
No. Observations:                2223   AIC:                                      4992.
Df Residuals:                    2213   BIC:                                      5049.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.0934      0.034      2.749      0.006       0.027       0.160
Hue               0.0906      0.017      5.383      0.000       0.058       0.124
Sat              -0.0581      0.025     -2.361      0.018      -0.106      -0.010
Lum              -0.4815      0.019    -25.026      0.000      -0.519      -0.444
sdHue            -0.0175      0.017     -1.030      0.303      -0.051       0.016
sdSat             0.0346      0.025      1.394      0.163      -0.014       0.083
sdBright          0.0085      0.020      0.422      0.673      -0.031       0.048
Entropy          -0.0209      0.021     -1.010      0.313      -0.062       0.020
StraightED        0.3155      0.036      8.754      0.000       0.245       0.386
NonStraightED     0.0446      0.045      0.993      0.321      -0.043       0.133
==============================================================================
Omnibus:                       23.093   Durbin-Watson:                   1.721
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.548
Skew:                           0.250   Prob(JB):                     7.70e-06
Kurtosis:                       3.063   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

####All of Transparency & All of Mid-Level Features (sans OK)

In [ ]:
run_reg(all_tr, all_lo_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Transparency   R-squared (uncentered):                   0.164
Model:                            OLS   Adj. R-squared (uncentered):              0.160
Method:                 Least Squares   F-statistic:                              43.41
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    3.99e-79
Time:                        17:10:39   Log-Likelihood:                         -2955.2
No. Observations:                2223   AIC:                                      5930.
Df Residuals:                    2213   BIC:                                      5988.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
ED                0.1223      0.042      2.914      0.004       0.040       0.205
Hue              -0.0358      0.021     -1.722      0.085      -0.077       0.005
Sat              -0.0844      0.030     -2.776      0.006      -0.144      -0.025
Lum              -0.0869      0.024     -3.657      0.000      -0.133      -0.040
sdHue             0.0790      0.021      3.772      0.000       0.038       0.120
sdSat             0.0834      0.031      2.720      0.007       0.023       0.144
sdBright         -0.0172      0.025     -0.695      0.487      -0.066       0.031
Entropy           0.0466      0.026      1.817      0.069      -0.004       0.097
StraightED        0.1726      0.045      3.879      0.000       0.085       0.260
NonStraightED    -0.2752      0.055     -4.964      0.000      -0.384      -0.166
==============================================================================
Omnibus:                       68.634   Durbin-Watson:                   1.694
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.185
Skew:                           0.313   Prob(JB):                     6.30e-13
Kurtosis:                       2.537   Cond. No.                         6.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Humanscale & All of Mid-Level Features (sans OK)

In [ ]:
run_reg(all_hs, all_mid_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Humanscale   R-squared (uncentered):                   0.197
Model:                            OLS   Adj. R-squared (uncentered):              0.190
Method:                 Least Squares   F-statistic:                              27.04
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    3.13e-90
Time:                        17:10:39   Log-Likelihood:                         -2910.3
No. Observations:                2223   AIC:                                      5861.
Df Residuals:                    2203   BIC:                                      5975.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road             -0.2151      0.035     -6.164      0.000      -0.284      -0.147
Sidewalk          0.0253      0.029      0.877      0.380      -0.031       0.082
Building         -0.2743      0.058     -4.756      0.000      -0.387      -0.161
Wall             -0.0557      0.026     -2.175      0.030      -0.106      -0.005
Fence            -0.0371      0.022     -1.704      0.089      -0.080       0.006
Pole              0.0018      0.023      0.078      0.938      -0.043       0.046
Traffic Light    -0.1332      0.021     -6.267      0.000      -0.175      -0.092
Traffic Sign      0.0546      0.021      2.653      0.008       0.014       0.095
Vegetation        0.0705      0.052      1.356      0.175      -0.031       0.173
Terrain          -0.0025      0.026     -0.100      0.921      -0.053       0.048
Sky               0.0157      0.052      0.304      0.761      -0.086       0.117
Person           -0.0251      0.020     -1.226      0.220      -0.065       0.015
Rider            -0.0381      0.021     -1.854      0.064      -0.078       0.002
Car               0.0892      0.029      3.128      0.002       0.033       0.145
Truck            -0.0005      0.020     -0.027      0.979      -0.040       0.039
Bus              -0.0332      0.020     -1.688      0.092      -0.072       0.005
Train            -0.0227      0.019     -1.171      0.242      -0.061       0.015
Motorcycle       -0.0014      0.019     -0.072      0.942      -0.039       0.036
Bicycle           0.0153      0.020      0.749      0.454      -0.025       0.055
Unlabeled        -0.1465      0.030     -4.917      0.000      -0.205      -0.088
==============================================================================
Omnibus:                       22.745   Durbin-Watson:                   1.578
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.436
Skew:                           0.224   Prob(JB):                     1.34e-05
Kurtosis:                       2.797   Cond. No.                         7.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### All of Disorder & All of Mid-Level Features (sans OK)

In [ ]:
run_reg(all_dis, all_mid_no_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               Disorder   R-squared (uncentered):                   0.295
Model:                            OLS   Adj. R-squared (uncentered):              0.289
Method:                 Least Squares   F-statistic:                              46.17
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   4.01e-151
Time:                        17:10:39   Log-Likelihood:                         -2765.2
No. Observations:                2223   AIC:                                      5570.
Df Residuals:                    2203   BIC:                                      5684.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road             -0.0258      0.033     -0.790      0.429      -0.090       0.038
Sidewalk         -0.0107      0.027     -0.397      0.691      -0.064       0.042
Building          0.1556      0.054      2.880      0.004       0.050       0.262
Wall             -0.0378      0.024     -1.576      0.115      -0.085       0.009
Fence             0.0139      0.020      0.681      0.496      -0.026       0.054
Pole             -0.0211      0.021     -0.987      0.324      -0.063       0.021
Traffic Light     0.0545      0.020      2.735      0.006       0.015       0.094
Traffic Sign      0.1067      0.019      5.535      0.000       0.069       0.145
Vegetation        0.0461      0.049      0.945      0.345      -0.050       0.142
Terrain          -0.0477      0.024     -1.994      0.046      -0.095      -0.001
Sky               0.3127      0.049      6.441      0.000       0.217       0.408
Person            0.1369      0.019      7.136      0.000       0.099       0.174
Rider            -0.0157      0.019     -0.815      0.415      -0.053       0.022
Car               0.4521      0.027     16.922      0.000       0.400       0.505
Truck             0.1249      0.019      6.684      0.000       0.088       0.161
Bus               0.0568      0.018      3.079      0.002       0.021       0.093
Train            -0.0037      0.018     -0.205      0.838      -0.039       0.032
Motorcycle        0.0142      0.018      0.792      0.428      -0.021       0.049
Bicycle           0.0012      0.019      0.063      0.950      -0.036       0.039
Unlabeled         0.1519      0.028      5.441      0.000       0.097       0.207
==============================================================================
Omnibus:                       26.451   Durbin-Watson:                   1.773
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.238
Skew:                           0.271   Prob(JB):                     1.22e-06
Kurtosis:                       2.975   Cond. No.                         7.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### OLS by Data

*   Sidewalk data
*   Road data
*   Emily Talen data
*   Img307/Omid Kardan data



#### Preference

In [ ]:
#low-sidewalk
run_reg(pref_sw, clean_lo_sw)

#mid-sidewalk
run_reg(pref_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Preference   R-squared (uncentered):                   0.545
Model:                            OLS   Adj. R-squared (uncentered):              0.528
Method:                 Least Squares   F-statistic:                              31.92
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    7.86e-78
Time:                        17:10:39   Log-Likelihood:                         -565.64
No. Observations:                 552   AIC:                                      1171.
Df Residuals:                     532   BIC:                                      1258.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -2.439e+05   1.56e+05     -1.560      0.119   -5.51e+05    6.32e+04
Sidewalk      -1.338e+05   8.58e+04     -1.560      0.119   -3.02e+05    3.47e+04
Building      -3.546e+05   2.27e+05     -1.560      0.119   -8.01e+05    9.19e+04
Wall           -5.21e+04   3.34e+04     -1.560      0.119   -1.18e+05    1.35e+04
Fence         -9.021e+04   5.78e+04     -1.560      0.119   -2.04e+05    2.34e+04
Pole          -2.551e+04   1.63e+04     -1.560      0.119   -5.76e+04    6611.237
Traffic Light -6368.6611   4082.371     -1.560      0.119   -1.44e+04    1650.883
Traffic Sign  -1.462e+04   9370.842     -1.560      0.119    -3.3e+04    3789.223
Vegetation    -3.187e+05   2.04e+05     -1.560      0.119    -7.2e+05    8.26e+04
Terrain       -1.504e+05   9.64e+04     -1.560      0.119    -3.4e+05     3.9e+04
Sky            -2.79e+05   1.79e+05     -1.560      0.119    -6.3e+05    7.23e+04
Person        -1.425e+04   9132.483     -1.560      0.119   -3.22e+04    3693.010
Rider         -1343.0058    860.884     -1.560      0.119   -3034.154     348.142
Car           -1.693e+05   1.09e+05     -1.560      0.119   -3.82e+05    4.39e+04
Truck         -4.606e+04   2.95e+04     -1.560      0.119   -1.04e+05    1.19e+04
Bus           -1.626e+04   1.04e+04     -1.560      0.119   -3.67e+04    4214.476
Train            -6.4455      4.141     -1.557      0.120     -14.580       1.689
Motorcycle    -5492.8984   3520.933     -1.560      0.119   -1.24e+04    1423.738
Bicycle       -8557.9389   5485.667     -1.560      0.119   -1.93e+04    2218.287
Unlabeled      -6.21e+04   3.98e+04     -1.560      0.119    -1.4e+05    1.61e+04
==============================================================================
Omnibus:                        9.038   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.011   Jarque-Bera (JB):                8.995
Skew:                           0.291   Prob(JB):                       0.0111
Kurtosis:                       3.227   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(pref_rd, clean_lo_rd)

#mid-road
run_reg(pref_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Preference   R-squared (uncentered):                   0.236
Model:                            OLS   Adj. R-squared (uncentered):              0.208
Method:                 Least Squares   F-statistic:                              8.646
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.67e-21
Time:                        17:10:39   Log-Likelihood:                         -709.11
No. Observations:                 552   AIC:                                      1456.
Df Residuals:                     533   BIC:                                      1538.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -1.376e+11   4.94e+12     -0.028      0.978   -9.85e+12    9.57e+12
Sidewalk      -7.325e+10   2.63e+12     -0.028      0.978   -5.24e+12     5.1e+12
Building      -2.488e+11   8.94e+12     -0.028      0.978   -1.78e+13    1.73e+13
Wall            -3.6e+10   1.29e+12     -0.028      0.978   -2.58e+12     2.5e+12
Fence         -3.874e+10   1.39e+12     -0.028      0.978   -2.77e+12     2.7e+12
Pole          -2.125e+10   7.63e+11     -0.028      0.978   -1.52e+12    1.48e+12
Traffic Light -4.678e+09   1.68e+11     -0.028      0.978   -3.35e+11    3.25e+11
Traffic Sign  -9.147e+09   3.29e+11     -0.028      0.978   -6.55e+11    6.36e+11
Vegetation    -2.903e+11   1.04e+13     -0.028      0.978   -2.08e+13    2.02e+13
Terrain       -7.733e+10   2.78e+12     -0.028      0.978   -5.53e+12    5.38e+12
Sky           -2.498e+11   8.98e+12     -0.028      0.978   -1.79e+13    1.74e+13
Person        -8.709e+09   3.13e+11     -0.028      0.978   -6.23e+11    6.06e+11
Rider         -8.758e+08   3.15e+10     -0.028      0.978   -6.27e+10    6.09e+10
Car           -9.818e+10   3.53e+12     -0.028      0.978   -7.03e+12    6.83e+12
Truck         -2.084e+10   7.49e+11     -0.028      0.978   -1.49e+12    1.45e+12
Bus            -1.99e+10   7.15e+11     -0.028      0.978   -1.42e+12    1.38e+12
Train         -1.956e+08   7.03e+09     -0.028      0.978    -1.4e+10    1.36e+10
Motorcycle     -1.12e+09   4.02e+10     -0.028      0.978   -8.01e+10    7.79e+10
Bicycle       -3.816e+09   1.37e+11     -0.028      0.978   -2.73e+11    2.65e+11
Unlabeled     -8.574e+10   3.08e+12     -0.028      0.978   -6.14e+12    5.97e+12
==============================================================================
Omnibus:                       14.691   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.469
Skew:                           0.361   Prob(JB):                     0.000721
Kurtosis:                       2.673   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(pref_et, clean_lo_et)

#mid-emily-talen
run_reg(pref_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Preference   R-squared (uncentered):                   0.427
Model:                            OLS   Adj. R-squared (uncentered):              0.416
Method:                 Least Squares   F-statistic:                              40.89
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   1.19e-117
Time:                        17:10:39   Log-Likelihood:                         -1276.6
No. Observations:                1119   AIC:                                      2593.
Df Residuals:                    1099   BIC:                                      2694.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           4.979e+04   6.27e+04      0.795      0.427   -7.32e+04    1.73e+05
Sidewalk       4.884e+04   6.15e+04      0.795      0.427   -7.18e+04    1.69e+05
Building       1.064e+05   1.34e+05      0.795      0.427   -1.56e+05    3.69e+05
Wall           2676.2857   3367.952      0.795      0.427   -3932.057    9284.628
Fence          1.224e+04   1.54e+04      0.795      0.427    -1.8e+04    4.25e+04
Pole           9691.1957   1.22e+04      0.795      0.427   -1.42e+04    3.36e+04
Traffic Light  2736.2210   3443.244      0.795      0.427   -4019.854    9492.296
Traffic Sign   5245.6060   6601.191      0.795      0.427   -7706.755    1.82e+04
Vegetation     7.202e+04   9.06e+04      0.795      0.427   -1.06e+05     2.5e+05
Terrain        1.231e+04   1.55e+04      0.795      0.427   -1.81e+04    4.27e+04
Sky            9.825e+04   1.24e+05      0.795      0.427   -1.44e+05    3.41e+05
Person         6515.5415   8199.279      0.795      0.427   -9572.469    2.26e+04
Rider           236.9395    298.131      0.795      0.427    -348.030     821.909
Car            3.187e+04   4.01e+04      0.795      0.427   -4.68e+04    1.11e+05
Truck          6051.8900   7615.943      0.795      0.427   -8891.541     2.1e+04
Bus            3677.0558   4627.352      0.795      0.427   -5402.387    1.28e+04
Train           117.9842    148.413      0.795      0.427    -173.220     409.188
Motorcycle      193.2375    243.148      0.795      0.427    -283.849     670.324
Bicycle        1757.0992   2211.221      0.795      0.427   -2581.592    6095.790
Unlabeled      6.391e+04   8.04e+04      0.795      0.427   -9.39e+04    2.22e+05
==============================================================================
Omnibus:                        4.477   Durbin-Watson:                   1.793
Prob(Omnibus):                  0.107   Jarque-Bera (JB):                4.222
Skew:                           0.106   Prob(JB):                        0.121
Kurtosis:                       2.787   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-OK
run_reg(pref_ok, clean_lo_ok)

#mid-OK
run_reg(pref_ok, clean_mid_ok)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Preference   R-squared (uncentered):                   0.053
Model:                            OLS   Adj. R-squared (uncentered):             -0.010
Method:                 Least Squares   F-statistic:                             0.8448
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                       0.652
Time:                        17:10:40   Log-Likelihood:                         -427.29
No. Observations:                 307   AIC:                                      892.6
Df Residuals:                     288   BIC:                                      963.4
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road              0.0604      0.068      0.888      0.375      -0.074       0.194
Sidewalk          0.1239      0.064      1.945      0.053      -0.001       0.249
Building         -0.0815      0.062     -1.312      0.190      -0.204       0.041
Wall              0.0033      0.068      0.049      0.961      -0.130       0.137
Fence             0.0095      0.055      0.171      0.864      -0.099       0.118
Pole              0.0596      0.067      0.896      0.371      -0.071       0.191
Traffic Light     0.0303      0.075      0.407      0.684      -0.116       0.177
Traffic Sign      0.0984      0.065      1.503      0.134      -0.030       0.227
Vegetation       -0.0274      0.039     -0.706      0.481      -0.104       0.049
Terrain           0.0291      0.053      0.553      0.581      -0.074       0.133
Sky              -0.0094      0.056     -0.168      0.867      -0.120       0.101
Person           -0.0271      0.063     -0.432      0.666      -0.151       0.096
Rider            -0.0478      0.059     -0.814      0.417      -0.164       0.068
Car               0.0455      0.069      0.662      0.508      -0.090       0.181
Truck             0.0202      0.058      0.349      0.727      -0.094       0.134
Bus              -0.0090      0.080     -0.112      0.911      -0.167       0.149
Train            -0.0173      0.059     -0.296      0.768      -0.132       0.098
Motorcycle       -0.0130      0.066     -0.196      0.845      -0.143       0.117
Bicycle          -0.0682      0.059     -1.156      0.249      -0.184       0.048
Unlabeled         0.0189      0.057      0.333      0.740      -0.093       0.131
==============================================================================
Omnibus:                        0.223   Durbin-Watson:                   0.766
Prob(Omnibus):                  0.895   Jarque-Bera (JB):                0.094
Skew:                          -0.030   Prob(JB):                        0.954
Kurtosis:                       3.061   Cond. No.                     5.44e+15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.15e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Walkability

In [ ]:
#low-sidewalk
run_reg(walk_sw, clean_lo_sw)

#mid-sidewalk
run_reg(walk_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            Walkability   R-squared (uncentered):                   0.605
Model:                            OLS   Adj. R-squared (uncentered):              0.590
Method:                 Least Squares   F-statistic:                              40.68
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.50e-93
Time:                        17:10:40   Log-Likelihood:                         -527.13
No. Observations:                 552   AIC:                                      1094.
Df Residuals:                     532   BIC:                                      1181.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -2.007e+05   1.46e+05     -1.377      0.169   -4.87e+05    8.57e+04
Sidewalk      -1.101e+05      8e+04     -1.377      0.169   -2.67e+05     4.7e+04
Building      -2.918e+05   2.12e+05     -1.377      0.169   -7.08e+05    1.25e+05
Wall          -4.288e+04   3.11e+04     -1.377      0.169   -1.04e+05    1.83e+04
Fence         -7.424e+04   5.39e+04     -1.377      0.169    -1.8e+05    3.17e+04
Pole          -2.099e+04   1.52e+04     -1.377      0.169   -5.09e+04    8963.442
Traffic Light -5240.9694   3807.285     -1.377      0.169   -1.27e+04    2238.187
Traffic Sign  -1.203e+04   8739.398     -1.377      0.169   -2.92e+04    5137.397
Vegetation    -2.623e+05   1.91e+05     -1.377      0.169   -6.37e+05    1.12e+05
Terrain       -1.238e+05   8.99e+04     -1.377      0.169      -3e+05    5.29e+04
Sky           -2.296e+05   1.67e+05     -1.377      0.169   -5.57e+05    9.81e+04
Person        -1.172e+04   8517.101     -1.377      0.169   -2.85e+04    5006.915
Rider         -1105.1864    802.874     -1.377      0.169   -2682.378     472.006
Car           -1.393e+05   1.01e+05     -1.377      0.169   -3.38e+05    5.95e+04
Truck          -3.79e+04   2.75e+04     -1.377      0.169    -9.2e+04    1.62e+04
Bus           -1.338e+04   9720.162     -1.377      0.169   -3.25e+04    5713.935
Train            -5.3281      3.862     -1.380      0.168     -12.915       2.258
Motorcycle    -4520.2786   3283.678     -1.377      0.169    -1.1e+04    1930.288
Bicycle       -7042.5895   5116.021     -1.377      0.169   -1.71e+04    3007.492
Unlabeled     -5.111e+04   3.71e+04     -1.377      0.169   -1.24e+05    2.18e+04
==============================================================================
Omnibus:                       11.067   Durbin-Watson:                   1.865
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               12.473
Skew:                           0.262   Prob(JB):                      0.00196
Kurtosis:                       3.518   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(walk_rd, clean_lo_rd)

#mid-road
run_reg(walk_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            Walkability   R-squared (uncentered):                   0.231
Model:                            OLS   Adj. R-squared (uncentered):              0.204
Method:                 Least Squares   F-statistic:                              8.425
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    7.07e-21
Time:                        17:10:40   Log-Likelihood:                         -710.77
No. Observations:                 552   AIC:                                      1460.
Df Residuals:                     533   BIC:                                      1541.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -5.165e+12   4.96e+12     -1.042      0.298   -1.49e+13    4.57e+12
Sidewalk       -2.75e+12   2.64e+12     -1.042      0.298   -7.94e+12    2.43e+12
Building      -9.343e+12   8.97e+12     -1.042      0.298    -2.7e+13    8.27e+12
Wall          -1.352e+12    1.3e+12     -1.042      0.298    -3.9e+12     1.2e+12
Fence         -1.455e+12    1.4e+12     -1.042      0.298    -4.2e+12    1.29e+12
Pole          -7.979e+11   7.66e+11     -1.042      0.298    -2.3e+12    7.06e+11
Traffic Light -1.757e+11   1.69e+11     -1.042      0.298   -5.07e+11    1.55e+11
Traffic Sign  -3.435e+11    3.3e+11     -1.042      0.298   -9.91e+11    3.04e+11
Vegetation     -1.09e+13   1.05e+13     -1.042      0.298   -3.14e+13    9.65e+12
Terrain       -2.904e+12   2.79e+12     -1.042      0.298   -8.38e+12    2.57e+12
Sky           -9.381e+12      9e+12     -1.042      0.298   -2.71e+13     8.3e+12
Person         -3.27e+11   3.14e+11     -1.042      0.298   -9.44e+11    2.89e+11
Rider         -3.289e+10   3.16e+10     -1.042      0.298   -9.49e+10    2.91e+10
Car           -3.687e+12   3.54e+12     -1.042      0.298   -1.06e+13    3.26e+12
Truck         -7.825e+11   7.51e+11     -1.042      0.298   -2.26e+12    6.93e+11
Bus           -7.471e+11   7.17e+11     -1.042      0.298   -2.16e+12    6.61e+11
Train         -7.344e+09   7.05e+09     -1.042      0.298   -2.12e+10     6.5e+09
Motorcycle    -4.204e+10   4.03e+10     -1.042      0.298   -1.21e+11    3.72e+10
Bicycle       -1.433e+11   1.37e+11     -1.042      0.298   -4.13e+11    1.27e+11
Unlabeled     -3.219e+12   3.09e+12     -1.042      0.298   -9.29e+12    2.85e+12
==============================================================================
Omnibus:                       21.481   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.439
Skew:                           0.399   Prob(JB):                     6.01e-05
Kurtosis:                       2.544   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(walk_et, clean_lo_et)

#mid-emily-talen
run_reg(walk_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            Walkability   R-squared (uncentered):                   0.482
Model:                            OLS   Adj. R-squared (uncentered):              0.473
Method:                 Least Squares   F-statistic:                              51.18
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   1.58e-141
Time:                        17:10:40   Log-Likelihood:                         -1219.5
No. Observations:                1119   AIC:                                      2479.
Df Residuals:                    1099   BIC:                                      2579.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           6.034e+04   5.95e+04      1.013      0.311   -5.65e+04    1.77e+05
Sidewalk       5.919e+04   5.84e+04      1.013      0.311   -5.54e+04    1.74e+05
Building        1.29e+05   1.27e+05      1.013      0.311   -1.21e+05    3.79e+05
Wall           3243.4065   3200.405      1.013      0.311   -3036.187    9523.001
Fence          1.483e+04   1.46e+04      1.013      0.311   -1.39e+04    4.36e+04
Pole           1.174e+04   1.16e+04      1.013      0.311    -1.1e+04    3.45e+04
Traffic Light  3316.0491   3271.951      1.013      0.311   -3103.928    9736.026
Traffic Sign   6357.1636   6272.798      1.013      0.311   -5950.849    1.87e+04
Vegetation     8.729e+04   8.61e+04      1.013      0.311   -8.17e+04    2.56e+05
Terrain        1.492e+04   1.47e+04      1.013      0.311    -1.4e+04    4.38e+04
Sky            1.191e+05   1.17e+05      1.013      0.311   -1.11e+05     3.5e+05
Person         7896.2752   7791.386      1.013      0.311   -7391.396    2.32e+04
Rider           287.1335    283.299      1.014      0.311    -268.735     843.002
Car            3.862e+04   3.81e+04      1.013      0.311   -3.62e+04    1.13e+05
Truck          7334.3327   7237.068      1.013      0.311   -6865.699    2.15e+04
Bus            4456.2712   4397.153      1.013      0.311   -4171.492    1.31e+04
Train           142.9698    141.029      1.014      0.311    -133.748     419.687
Motorcycle      234.1725    231.052      1.014      0.311    -219.180     687.525
Bicycle        2129.4854   2101.218      1.013      0.311   -1993.366    6252.337
Unlabeled      7.745e+04   7.64e+04      1.013      0.311   -7.25e+04    2.27e+05
==============================================================================
Omnibus:                        1.816   Durbin-Watson:                   1.783
Prob(Omnibus):                  0.403   Jarque-Bera (JB):                1.699
Skew:                           0.064   Prob(JB):                        0.428
Kurtosis:                       3.141   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Imageability

In [ ]:
#low-sidewalk
run_reg(img_sw, clean_lo_sw)

#mid-sidewalk
run_reg(img_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Imageability   R-squared (uncentered):                   0.392
Model:                            OLS   Adj. R-squared (uncentered):              0.369
Method:                 Least Squares   F-statistic:                              17.12
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    2.00e-45
Time:                        17:10:40   Log-Likelihood:                         -646.13
No. Observations:                 552   AIC:                                      1332.
Df Residuals:                     532   BIC:                                      1419.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -3.482e+05   1.81e+05     -1.925      0.055   -7.04e+05    7115.656
Sidewalk       -1.91e+05   9.92e+04     -1.925      0.055   -3.86e+05    3903.245
Building      -5.063e+05   2.63e+05     -1.925      0.055   -1.02e+06    1.03e+04
Wall          -7.439e+04   3.86e+04     -1.925      0.055    -1.5e+05    1519.657
Fence         -1.288e+05   6.69e+04     -1.925      0.055    -2.6e+05    2631.598
Pole          -3.642e+04   1.89e+04     -1.925      0.055   -7.36e+04     744.090
Traffic Light -9092.6488   4723.258     -1.925      0.055   -1.84e+04     185.875
Traffic Sign  -2.087e+04   1.08e+04     -1.925      0.055   -4.22e+04     426.423
Vegetation     -4.55e+05   2.36e+05     -1.925      0.055   -9.19e+05    9297.520
Terrain       -2.148e+05   1.12e+05     -1.925      0.055   -4.34e+05    4388.058
Sky           -3.984e+05   2.07e+05     -1.925      0.055   -8.05e+05    8139.615
Person        -2.034e+04   1.06e+04     -1.925      0.055   -4.11e+04     415.772
Rider         -1917.4125    996.033     -1.925      0.055   -3874.052      39.227
Car           -2.417e+05   1.26e+05     -1.925      0.055   -4.88e+05    4938.599
Truck         -6.576e+04   3.42e+04     -1.925      0.055   -1.33e+05    1343.561
Bus           -2.321e+04   1.21e+04     -1.925      0.055   -4.69e+04     474.291
Train            -9.2144      4.791     -1.923      0.055     -18.626       0.197
Motorcycle    -7842.2921   4073.680     -1.925      0.055   -1.58e+04     160.180
Bicycle       -1.222e+04   6346.855     -1.925      0.055   -2.47e+04     249.725
Unlabeled     -8.867e+04   4.61e+04     -1.925      0.055   -1.79e+05    1811.769
==============================================================================
Omnibus:                       22.310   Durbin-Watson:                   1.816
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.918
Skew:                           0.487   Prob(JB):                     6.40e-06
Kurtosis:                       3.305   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(img_rd, clean_lo_rd)

#mid-road
run_reg(img_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Imageability   R-squared (uncentered):                   0.170
Model:                            OLS   Adj. R-squared (uncentered):              0.141
Method:                 Least Squares   F-statistic:                              5.756
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    3.48e-13
Time:                        17:10:40   Log-Likelihood:                         -731.74
No. Observations:                 552   AIC:                                      1501.
Df Residuals:                     533   BIC:                                      1583.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -1.784e+11   5.15e+12     -0.035      0.972   -1.03e+13    9.94e+12
Sidewalk      -9.501e+10   2.74e+12     -0.035      0.972   -5.48e+12    5.29e+12
Building      -3.228e+11   9.31e+12     -0.035      0.972   -1.86e+13     1.8e+13
Wall           -4.67e+10   1.35e+12     -0.035      0.972   -2.69e+12     2.6e+12
Fence         -5.025e+10   1.45e+12     -0.035      0.972    -2.9e+12     2.8e+12
Pole          -2.756e+10   7.95e+11     -0.035      0.972   -1.59e+12    1.53e+12
Traffic Light -6.068e+09   1.75e+11     -0.035      0.972    -3.5e+11    3.38e+11
Traffic Sign  -1.186e+10   3.42e+11     -0.035      0.972   -6.84e+11    6.61e+11
Vegetation    -3.765e+11   1.09e+13     -0.035      0.972   -2.17e+13     2.1e+13
Terrain       -1.003e+11   2.89e+12     -0.035      0.972   -5.79e+12    5.59e+12
Sky           -3.241e+11   9.35e+12     -0.035      0.972   -1.87e+13     1.8e+13
Person         -1.13e+10   3.26e+11     -0.035      0.972   -6.52e+11    6.29e+11
Rider         -1.136e+09   3.28e+10     -0.035      0.972   -6.55e+10    6.33e+10
Car           -1.274e+11   3.68e+12     -0.035      0.972   -7.35e+12    7.09e+12
Truck         -2.703e+10    7.8e+11     -0.035      0.972   -1.56e+12    1.51e+12
Bus           -2.581e+10   7.45e+11     -0.035      0.972   -1.49e+12    1.44e+12
Train         -2.537e+08   7.32e+09     -0.035      0.972   -1.46e+10    1.41e+10
Motorcycle    -1.452e+09   4.19e+10     -0.035      0.972   -8.38e+10    8.09e+10
Bicycle       -4.949e+09   1.43e+11     -0.035      0.972   -2.86e+11    2.76e+11
Unlabeled     -1.112e+11   3.21e+12     -0.035      0.972   -6.42e+12    6.19e+12
==============================================================================
Omnibus:                       15.382   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               13.826
Skew:                           0.328   Prob(JB):                     0.000995
Kurtosis:                       2.587   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(img_et, clean_lo_et)

#mid-emily-talen
run_reg(img_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Imageability   R-squared (uncentered):                   0.580
Model:                            OLS   Adj. R-squared (uncentered):              0.573
Method:                 Least Squares   F-statistic:                              76.02
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   5.41e-191
Time:                        17:10:40   Log-Likelihood:                         -1101.8
No. Observations:                1119   AIC:                                      2244.
Df Residuals:                    1099   BIC:                                      2344.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -2.827e+04   5.36e+04     -0.527      0.598   -1.33e+05    7.69e+04
Sidewalk      -2.772e+04   5.26e+04     -0.527      0.598   -1.31e+05    7.54e+04
Building      -6.041e+04   1.15e+05     -0.527      0.598   -2.85e+05    1.64e+05
Wall          -1519.3172   2880.971     -0.527      0.598   -7172.143    4133.508
Fence         -6949.0350   1.32e+04     -0.527      0.598   -3.28e+04    1.89e+04
Pole          -5501.6198   1.04e+04     -0.527      0.598    -2.6e+04     1.5e+04
Traffic Light -1553.1865   2945.376     -0.527      0.598   -7332.383    4226.010
Traffic Sign  -2977.7872   5646.708     -0.527      0.598   -1.41e+04    8101.759
Vegetation    -4.089e+04   7.75e+04     -0.527      0.598   -1.93e+05    1.11e+05
Terrain       -6990.6050   1.33e+04     -0.527      0.598    -3.3e+04     1.9e+04
Sky           -5.578e+04   1.06e+05     -0.527      0.598   -2.63e+05    1.52e+05
Person        -3698.6931   7013.725     -0.527      0.598   -1.75e+04    1.01e+04
Rider          -134.4848    255.023     -0.527      0.598    -634.872     365.902
Car           -1.809e+04   3.43e+04     -0.527      0.598   -8.54e+04    4.92e+04
Truck         -3435.5995   6514.734     -0.527      0.598   -1.62e+04    9347.122
Bus           -2087.4182   3958.272     -0.527      0.598   -9854.041    5679.205
Train           -66.9120    126.953     -0.527      0.598    -316.010     182.186
Motorcycle     -109.7161    207.990     -0.528      0.598    -517.819     298.387
Bicycle        -997.4917   1891.494     -0.527      0.598   -4708.840    2713.857
Unlabeled     -3.628e+04   6.88e+04     -0.527      0.598   -1.71e+05    9.87e+04
==============================================================================
Omnibus:                        2.799   Durbin-Watson:                   1.763
Prob(Omnibus):                  0.247   Jarque-Bera (JB):                2.665
Skew:                           0.110   Prob(JB):                        0.264
Kurtosis:                       3.094   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Complexity

In [ ]:
#low-sidewalk
run_reg(comp_sw, clean_lo_sw)

#mid-sidewalk
run_reg(comp_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Complexity   R-squared (uncentered):                   0.390
Model:                            OLS   Adj. R-squared (uncentered):              0.367
Method:                 Least Squares   F-statistic:                              16.99
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    4.27e-45
Time:                        17:10:40   Log-Likelihood:                         -646.96
No. Observations:                 552   AIC:                                      1334.
Df Residuals:                     532   BIC:                                      1420.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -1.301e+05   1.81e+05     -0.718      0.473   -4.86e+05    2.26e+05
Sidewalk      -7.137e+04   9.94e+04     -0.718      0.473   -2.67e+05    1.24e+05
Building      -1.892e+05   2.63e+05     -0.718      0.473   -7.07e+05    3.28e+05
Wall          -2.779e+04   3.87e+04     -0.718      0.473   -1.04e+05    4.82e+04
Fence         -4.812e+04    6.7e+04     -0.718      0.473    -1.8e+05    8.35e+04
Pole          -1.361e+04   1.89e+04     -0.718      0.473   -5.08e+04    2.36e+04
Traffic Light -3396.9789   4730.356     -0.718      0.473   -1.27e+04    5895.489
Traffic Sign  -7797.8394   1.09e+04     -0.718      0.473   -2.91e+04    1.35e+04
Vegetation      -1.7e+05   2.37e+05     -0.718      0.473   -6.35e+05    2.95e+05
Terrain       -8.024e+04   1.12e+05     -0.718      0.473      -3e+05    1.39e+05
Sky           -1.488e+05   2.07e+05     -0.718      0.473   -5.56e+05    2.58e+05
Person        -7599.2406   1.06e+04     -0.718      0.473   -2.84e+04    1.32e+04
Rider          -716.3236    997.530     -0.718      0.473   -2675.904    1243.257
Car            -9.03e+04   1.26e+05     -0.718      0.473   -3.37e+05    1.57e+05
Truck         -2.457e+04   3.42e+04     -0.718      0.473   -9.18e+04    4.26e+04
Bus           -8672.9584   1.21e+04     -0.718      0.473   -3.24e+04    1.51e+04
Train            -3.4756      4.798     -0.724      0.469     -12.901       5.950
Motorcycle    -2929.9191   4079.802     -0.718      0.473   -1.09e+04    5084.579
Bicycle       -4564.7296   6356.394     -0.718      0.473   -1.71e+04    7921.981
Unlabeled     -3.313e+04   4.61e+04     -0.718      0.473   -1.24e+05    5.75e+04
==============================================================================
Omnibus:                       24.058   Durbin-Watson:                   1.815
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               26.208
Skew:                           0.495   Prob(JB):                     2.04e-06
Kurtosis:                       3.399   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(comp_rd, clean_lo_rd)

#mid-road
run_reg(comp_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Complexity   R-squared (uncentered):                   0.174
Model:                            OLS   Adj. R-squared (uncentered):              0.145
Method:                 Least Squares   F-statistic:                              5.913
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.22e-13
Time:                        17:10:40   Log-Likelihood:                         -730.46
No. Observations:                 552   AIC:                                      1499.
Df Residuals:                     533   BIC:                                      1581.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           2.315e+12   5.14e+12      0.451      0.652   -7.78e+12    1.24e+13
Sidewalk       1.233e+12   2.74e+12      0.451      0.652   -4.14e+12    6.61e+12
Building       4.188e+12   9.29e+12      0.451      0.652   -1.41e+13    2.24e+13
Wall            6.06e+11   1.34e+12      0.451      0.652   -2.04e+12    3.25e+12
Fence           6.52e+11   1.45e+12      0.451      0.652   -2.19e+12    3.49e+12
Pole           3.577e+11   7.94e+11      0.451      0.652    -1.2e+12    1.92e+12
Traffic Light  7.874e+10   1.75e+11      0.451      0.652   -2.64e+11    4.22e+11
Traffic Sign    1.54e+11   3.42e+11      0.451      0.652   -5.17e+11    8.25e+11
Vegetation     4.886e+12   1.08e+13      0.451      0.652   -1.64e+13    2.62e+13
Terrain        1.302e+12   2.89e+12      0.451      0.652   -4.37e+12    6.97e+12
Sky            4.205e+12   9.33e+12      0.451      0.652   -1.41e+13    2.25e+13
Person         1.466e+11   3.25e+11      0.451      0.652   -4.92e+11    7.85e+11
Rider          1.474e+10   3.27e+10      0.451      0.652   -4.95e+10     7.9e+10
Car            1.653e+12   3.67e+12      0.451      0.652   -5.55e+12    8.86e+12
Truck          3.508e+11   7.78e+11      0.451      0.652   -1.18e+12    1.88e+12
Bus            3.349e+11   7.43e+11      0.451      0.652   -1.12e+12    1.79e+12
Train          3.292e+09    7.3e+09      0.451      0.652   -1.11e+10    1.76e+10
Motorcycle     1.884e+10   4.18e+10      0.451      0.652   -6.33e+10    1.01e+11
Bicycle        6.423e+10   1.42e+11      0.451      0.652   -2.16e+11    3.44e+11
Unlabeled      1.443e+12    3.2e+12      0.451      0.652   -4.85e+12    7.73e+12
==============================================================================
Omnibus:                        8.607   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.014   Jarque-Bera (JB):                8.060
Skew:                           0.248   Prob(JB):                       0.0178
Kurtosis:                       2.678   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(comp_et, clean_lo_et)

#mid-emily-talen
run_reg(comp_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Complexity   R-squared (uncentered):                   0.498
Model:                            OLS   Adj. R-squared (uncentered):              0.489
Method:                 Least Squares   F-statistic:                              54.56
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   7.20e-149
Time:                        17:10:40   Log-Likelihood:                         -1202.0
No. Observations:                1119   AIC:                                      2444.
Df Residuals:                    1099   BIC:                                      2544.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           8.251e+04   5.86e+04      1.408      0.160   -3.25e+04    1.98e+05
Sidewalk       8.093e+04   5.75e+04      1.408      0.160   -3.19e+04    1.94e+05
Building       1.763e+05   1.25e+05      1.408      0.160   -6.95e+04    4.22e+05
Wall           4434.7378   3150.722      1.408      0.160   -1747.373    1.06e+04
Fence          2.028e+04   1.44e+04      1.408      0.160   -7992.029    4.86e+04
Pole           1.606e+04   1.14e+04      1.408      0.160   -6327.369    3.84e+04
Traffic Light  4534.1121   3221.158      1.408      0.160   -1786.202    1.09e+04
Traffic Sign   8692.2678   6175.420      1.408      0.160   -3424.678    2.08e+04
Vegetation     1.193e+05   8.48e+04      1.408      0.160    -4.7e+04    2.86e+05
Terrain         2.04e+04   1.45e+04      1.408      0.160   -8039.840    4.88e+04
Sky            1.628e+05   1.16e+05      1.408      0.160   -6.41e+04     3.9e+05
Person          1.08e+04   7670.434      1.408      0.160   -4253.726    2.58e+04
Rider           392.5663    278.901      1.408      0.160    -154.673     939.806
Car            5.281e+04   3.75e+04      1.408      0.160   -2.08e+04    1.26e+05
Truck          1.003e+04   7124.721      1.408      0.160   -3951.247     2.4e+04
Bus            6093.1295   4328.892      1.408      0.160   -2400.698    1.46e+04
Train           195.4275    138.840      1.408      0.160     -76.994     467.849
Motorcycle      320.1574    227.465      1.408      0.160    -126.157     766.472
Bicycle        2911.6546   2068.599      1.408      0.160   -1147.195    6970.504
Unlabeled      1.059e+05   7.52e+04      1.408      0.160   -4.17e+04    2.54e+05
==============================================================================
Omnibus:                        6.646   Durbin-Watson:                   1.636
Prob(Omnibus):                  0.036   Jarque-Bera (JB):                6.720
Skew:                           0.189   Prob(JB):                       0.0347
Kurtosis:                       2.958   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Enclosure

In [ ]:
#low-sidewalk
run_reg(en_sw, clean_lo_sw)

#mid-sidewalk
run_reg(en_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Enclosure   R-squared (uncentered):                   0.271
Model:                            OLS   Adj. R-squared (uncentered):              0.243
Method:                 Least Squares   F-statistic:                              9.867
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    6.61e-26
Time:                        17:10:40   Log-Likelihood:                         -696.17
No. Observations:                 552   AIC:                                      1432.
Df Residuals:                     532   BIC:                                      1519.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           1.573e+05   1.98e+05      0.794      0.427   -2.32e+05    5.46e+05
Sidewalk       8.628e+04   1.09e+05      0.794      0.427   -1.27e+05       3e+05
Building       2.287e+05   2.88e+05      0.794      0.427   -3.37e+05    7.94e+05
Wall            3.36e+04   4.23e+04      0.794      0.427   -4.95e+04    1.17e+05
Fence          5.817e+04   7.33e+04      0.794      0.427   -8.57e+04    2.02e+05
Pole           1.645e+04   2.07e+04      0.794      0.427   -2.42e+04    5.71e+04
Traffic Light  4106.6470   5171.443      0.794      0.427   -6052.307    1.43e+04
Traffic Sign   9426.5712   1.19e+04      0.794      0.427   -1.39e+04    3.27e+04
Vegetation     2.055e+05   2.59e+05      0.794      0.427   -3.03e+05    7.14e+05
Terrain          9.7e+04   1.22e+05      0.794      0.427   -1.43e+05    3.37e+05
Sky            1.799e+05   2.27e+05      0.794      0.427   -2.65e+05    6.25e+05
Person         9186.9088   1.16e+04      0.794      0.427   -1.35e+04    3.19e+04
Rider           866.0431   1090.545      0.794      0.427   -1276.260    3008.346
Car            1.092e+05   1.37e+05      0.794      0.427   -1.61e+05    3.79e+05
Truck           2.97e+04   3.74e+04      0.794      0.427   -4.38e+04    1.03e+05
Bus            1.048e+04   1.32e+04      0.794      0.427   -1.55e+04    3.64e+04
Train             4.1747      5.246      0.796      0.426      -6.130      14.479
Motorcycle     3541.9302   4460.227      0.794      0.427   -5219.888    1.23e+04
Bicycle        5518.3840   6949.102      0.794      0.427   -8132.663    1.92e+04
Unlabeled      4.005e+04   5.04e+04      0.794      0.427    -5.9e+04    1.39e+05
==============================================================================
Omnibus:                       14.538   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               13.712
Skew:                           0.339   Prob(JB):                      0.00105
Kurtosis:                       2.630   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(en_rd, clean_lo_rd)

#mid-road
run_reg(en_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Enclosure   R-squared (uncentered):                   0.733
Model:                            OLS   Adj. R-squared (uncentered):              0.724
Method:                 Least Squares   F-statistic:                              77.10
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   3.42e-139
Time:                        17:10:40   Log-Likelihood:                         -418.56
No. Observations:                 552   AIC:                                      875.1
Df Residuals:                     533   BIC:                                      957.1
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           2.876e+12   2.92e+12      0.985      0.325   -2.86e+12    8.61e+12
Sidewalk       1.532e+12   1.55e+12      0.985      0.325   -1.52e+12    4.59e+12
Building       5.203e+12   5.28e+12      0.985      0.325   -5.17e+12    1.56e+13
Wall           7.527e+11   7.64e+11      0.985      0.325   -7.48e+11    2.25e+12
Fence          8.099e+11   8.22e+11      0.985      0.325   -8.05e+11    2.43e+12
Pole           4.443e+11   4.51e+11      0.985      0.325   -4.42e+11    1.33e+12
Traffic Light  9.781e+10   9.93e+10      0.985      0.325   -9.72e+10    2.93e+11
Traffic Sign   1.913e+11   1.94e+11      0.985      0.325    -1.9e+11    5.73e+11
Vegetation      6.07e+12   6.16e+12      0.985      0.325   -6.03e+12    1.82e+13
Terrain        1.617e+12   1.64e+12      0.985      0.325   -1.61e+12    4.84e+12
Sky            5.224e+12    5.3e+12      0.985      0.325   -5.19e+12    1.56e+13
Person         1.821e+11   1.85e+11      0.985      0.325   -1.81e+11    5.45e+11
Rider          1.831e+10   1.86e+10      0.985      0.325   -1.82e+10    5.48e+10
Car            2.053e+12   2.08e+12      0.985      0.325   -2.04e+12    6.15e+12
Truck          4.357e+11   4.42e+11      0.985      0.325   -4.33e+11     1.3e+12
Bus             4.16e+11   4.22e+11      0.985      0.325   -4.14e+11    1.25e+12
Train          4.089e+09   4.15e+09      0.985      0.325   -4.06e+09    1.22e+10
Motorcycle     2.341e+10   2.38e+10      0.985      0.325   -2.33e+10    7.01e+10
Bicycle        7.978e+10    8.1e+10      0.985      0.325   -7.93e+10    2.39e+11
Unlabeled      1.793e+12   1.82e+12      0.985      0.325   -1.78e+12    5.37e+12
==============================================================================
Omnibus:                        1.146   Durbin-Watson:                   1.713
Prob(Omnibus):                  0.564   Jarque-Bera (JB):                0.963
Skew:                          -0.047   Prob(JB):                        0.618
Kurtosis:                       3.181   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(en_et, clean_lo_et)

#mid-emily-talen
run_reg(en_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Enclosure   R-squared (uncentered):                   0.735
Model:                            OLS   Adj. R-squared (uncentered):              0.730
Method:                 Least Squares   F-statistic:                              152.6
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                   6.48e-300
Time:                        17:10:41   Log-Likelihood:                         -844.30
No. Observations:                1119   AIC:                                      1729.
Df Residuals:                    1099   BIC:                                      1829.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road          -1.201e+04   4.26e+04     -0.282      0.778   -9.56e+04    7.15e+04
Sidewalk      -1.178e+04   4.18e+04     -0.282      0.778   -9.37e+04    7.02e+04
Building      -2.567e+04    9.1e+04     -0.282      0.778   -2.04e+05    1.53e+05
Wall           -645.4543   2288.712     -0.282      0.778   -5136.192    3845.284
Fence         -2952.1861   1.05e+04     -0.282      0.778   -2.35e+04    1.76e+04
Pole          -2337.3499   8287.672     -0.282      0.778   -1.86e+04    1.39e+04
Traffic Light  -659.8677   2339.877     -0.282      0.778   -5250.998    3931.263
Traffic Sign  -1265.0818   4485.878     -0.282      0.778   -1.01e+04    7536.771
Vegetation    -1.737e+04   6.16e+04     -0.282      0.778   -1.38e+05    1.03e+05
Terrain       -2969.9067   1.05e+04     -0.282      0.778   -2.36e+04    1.77e+04
Sky            -2.37e+04    8.4e+04     -0.282      0.778   -1.89e+05    1.41e+05
Person        -1571.3412   5571.869     -0.282      0.778   -1.25e+04    9361.361
Rider           -57.1418    202.596     -0.282      0.778    -454.661     340.378
Car           -7686.4251   2.73e+04     -0.282      0.778   -6.12e+04    4.58e+04
Truck         -1459.5088   5175.459     -0.282      0.778   -1.16e+04    8695.387
Bus            -886.8097   3144.544     -0.282      0.778   -7056.799    5283.179
Train           -28.4149    100.855     -0.282      0.778    -226.304     169.475
Motorcycle      -46.5985    165.233     -0.282      0.778    -370.805     277.608
Bicycle        -423.7699   1502.648     -0.282      0.778   -3372.153    2524.613
Unlabeled     -1.541e+04   5.47e+04     -0.282      0.778   -1.23e+05    9.18e+04
==============================================================================
Omnibus:                        0.157   Durbin-Watson:                   1.701
Prob(Omnibus):                  0.925   Jarque-Bera (JB):                0.207
Skew:                          -0.024   Prob(JB):                        0.902
Kurtosis:                       2.954   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Transparency

In [ ]:
#low-sidewalk
run_reg(tr_sw, clean_lo_sw)

#mid-sidewalk
run_reg(tr_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Transparency   R-squared (uncentered):                   0.486
Model:                            OLS   Adj. R-squared (uncentered):              0.467
Method:                 Least Squares   F-statistic:                              25.18
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    3.92e-64
Time:                        17:10:41   Log-Likelihood:                         -599.43
No. Observations:                 552   AIC:                                      1239.
Df Residuals:                     532   BIC:                                      1325.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           1.351e+05   1.66e+05      0.813      0.417   -1.91e+05    4.62e+05
Sidewalk       7.413e+04   9.12e+04      0.813      0.417   -1.05e+05    2.53e+05
Building       1.965e+05   2.42e+05      0.813      0.417   -2.78e+05    6.71e+05
Wall           2.887e+04   3.55e+04      0.813      0.417   -4.09e+04    9.86e+04
Fence          4.998e+04   6.15e+04      0.813      0.417   -7.08e+04    1.71e+05
Pole           1.413e+04   1.74e+04      0.813      0.417      -2e+04    4.83e+04
Traffic Light  3528.3781   4340.077      0.813      0.417   -4997.412    1.21e+04
Traffic Sign   8099.1813   9962.390      0.813      0.417   -1.15e+04    2.77e+04
Vegetation     1.766e+05   2.17e+05      0.813      0.417    -2.5e+05    6.03e+05
Terrain        8.334e+04   1.03e+05      0.813      0.417   -1.18e+05    2.85e+05
Sky            1.546e+05    1.9e+05      0.813      0.417   -2.19e+05    5.28e+05
Person         7893.2958   9708.985      0.813      0.417   -1.12e+04     2.7e+04
Rider           744.0611    915.228      0.813      0.417   -1053.843    2541.966
Car            9.379e+04   1.15e+05      0.813      0.417   -1.33e+05     3.2e+05
Truck          2.552e+04   3.14e+04      0.813      0.417   -3.61e+04    8.72e+04
Bus            9008.0276   1.11e+04      0.813      0.417   -1.28e+04    3.08e+04
Train             3.5547      4.402      0.807      0.420      -5.093      12.203
Motorcycle     3043.1087   3743.197      0.813      0.417   -4310.151    1.04e+04
Bicycle        4741.2767   5831.958      0.813      0.417   -6715.214    1.62e+04
Unlabeled      3.441e+04   4.23e+04      0.813      0.417   -4.87e+04    1.18e+05
==============================================================================
Omnibus:                       11.912   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.003   Jarque-Bera (JB):                9.247
Skew:                           0.221   Prob(JB):                      0.00982
Kurtosis:                       2.545   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(tr_rd, clean_lo_rd)

#mid-road
run_reg(tr_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Transparency   R-squared (uncentered):                   0.213
Model:                            OLS   Adj. R-squared (uncentered):              0.185
Method:                 Least Squares   F-statistic:                              7.584
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.79e-18
Time:                        17:10:41   Log-Likelihood:                         -717.21
No. Observations:                 552   AIC:                                      1472.
Df Residuals:                     533   BIC:                                      1554.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           5.189e+12   5.02e+12      1.035      0.301   -4.66e+12     1.5e+13
Sidewalk       2.763e+12   2.67e+12      1.035      0.301   -2.48e+12    8.01e+12
Building       9.386e+12   9.07e+12      1.035      0.301   -8.44e+12    2.72e+13
Wall           1.358e+12   1.31e+12      1.035      0.301   -1.22e+12    3.94e+12
Fence          1.461e+12   1.41e+12      1.035      0.301   -1.31e+12    4.24e+12
Pole           8.015e+11   7.75e+11      1.035      0.301    -7.2e+11    2.32e+12
Traffic Light  1.765e+11   1.71e+11      1.035      0.301   -1.59e+11    5.11e+11
Traffic Sign    3.45e+11   3.33e+11      1.035      0.301    -3.1e+11       1e+12
Vegetation     1.095e+13   1.06e+13      1.035      0.301   -9.84e+12    3.17e+13
Terrain        2.917e+12   2.82e+12      1.035      0.301   -2.62e+12    8.46e+12
Sky            9.424e+12   9.11e+12      1.035      0.301   -8.47e+12    2.73e+13
Person         3.285e+11   3.18e+11      1.035      0.301   -2.95e+11    9.52e+11
Rider          3.304e+10   3.19e+10      1.035      0.301   -2.97e+10    9.58e+10
Car            3.704e+12   3.58e+12      1.035      0.301   -3.33e+12    1.07e+13
Truck          7.861e+11    7.6e+11      1.035      0.301   -7.06e+11    2.28e+12
Bus            7.505e+11   7.25e+11      1.035      0.301   -6.74e+11    2.18e+12
Train          7.377e+09   7.13e+09      1.035      0.301   -6.63e+09    2.14e+10
Motorcycle     4.223e+10   4.08e+10      1.035      0.301   -3.79e+10    1.22e+11
Bicycle        1.439e+11   1.39e+11      1.035      0.301   -1.29e+11    4.17e+11
Unlabeled      3.234e+12   3.13e+12      1.035      0.301   -2.91e+12    9.37e+12
==============================================================================
Omnibus:                       22.016   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.293
Skew:                           0.413   Prob(JB):                     3.92e-05
Kurtosis:                       2.553   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(tr_et, clean_lo_et)

#mid-emily-talen
run_reg(tr_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Transparency   R-squared (uncentered):                   0.359
Model:                            OLS   Adj. R-squared (uncentered):              0.347
Method:                 Least Squares   F-statistic:                              30.79
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    9.41e-92
Time:                        17:10:41   Log-Likelihood:                         -1338.9
No. Observations:                1119   AIC:                                      2718.
Df Residuals:                    1099   BIC:                                      2818.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           1.704e+05   6.62e+04      2.572      0.010    4.04e+04       3e+05
Sidewalk       1.671e+05    6.5e+04      2.572      0.010    3.96e+04    2.95e+05
Building       3.641e+05   1.42e+05      2.572      0.010    8.63e+04    6.42e+05
Wall           9157.4648   3560.742      2.572      0.010    2170.845    1.61e+04
Fence          4.188e+04   1.63e+04      2.572      0.010    9929.189    7.38e+04
Pole           3.316e+04   1.29e+04      2.572      0.010    7861.159    5.85e+04
Traffic Light  9362.3515   3640.343      2.572      0.010    2219.543    1.65e+04
Traffic Sign   1.795e+04   6979.059      2.572      0.010    4255.134    3.16e+04
Vegetation     2.464e+05   9.58e+04      2.572      0.010    5.84e+04    4.34e+05
Terrain        4.213e+04   1.64e+04      2.572      0.010    9988.674    7.43e+04
Sky            3.362e+05   1.31e+05      2.572      0.010    7.97e+04    5.93e+05
Person         2.229e+04   8668.625      2.572      0.010    5285.185    3.93e+04
Rider           810.5796    315.196      2.572      0.010     192.125    1429.034
Car            1.091e+05   4.24e+04      2.572      0.010    2.59e+04    1.92e+05
Truck          2.071e+04   8051.897      2.572      0.010    4909.060    3.65e+04
Bus            1.258e+04   4892.233      2.572      0.010    2982.700    2.22e+04
Train           403.5089    156.908      2.572      0.010      95.636     711.382
Motorcycle      661.0917    257.066      2.572      0.010     156.696    1165.488
Bicycle        6012.3833   2337.796      2.572      0.010    1425.335    1.06e+04
Unlabeled      2.187e+05    8.5e+04      2.572      0.010    5.18e+04    3.85e+05
==============================================================================
Omnibus:                       15.020   Durbin-Watson:                   1.689
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.270
Skew:                           0.272   Prob(JB):                     0.000483
Kurtosis:                       2.824   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Humanscale

In [ ]:
#low-sidewalk
run_reg(hs_sw, clean_lo_sw)

#mid-sidewalk
run_reg(hs_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Humanscale   R-squared (uncentered):                   0.169
Model:                            OLS   Adj. R-squared (uncentered):              0.137
Method:                 Least Squares   F-statistic:                              5.394
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.34e-12
Time:                        17:10:41   Log-Likelihood:                         -732.30
No. Observations:                 552   AIC:                                      1505.
Df Residuals:                     532   BIC:                                      1591.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           9.199e+04   2.11e+05      0.435      0.664   -3.23e+05    5.07e+05
Sidewalk       5.046e+04   1.16e+05      0.435      0.664   -1.77e+05    2.78e+05
Building       1.337e+05   3.07e+05      0.435      0.664    -4.7e+05    7.38e+05
Wall           1.965e+04   4.52e+04      0.435      0.664   -6.91e+04    1.08e+05
Fence          3.402e+04   7.82e+04      0.435      0.664    -1.2e+05    1.88e+05
Pole           9618.9649   2.21e+04      0.435      0.664   -3.38e+04    5.31e+04
Traffic Light  2401.7612   5521.202      0.435      0.664   -8444.272    1.32e+04
Traffic Sign   5513.2012   1.27e+04      0.435      0.664   -1.94e+04    3.04e+04
Vegetation     1.202e+05   2.76e+05      0.435      0.664   -4.23e+05    6.63e+05
Terrain        5.673e+04    1.3e+05      0.435      0.664   -1.99e+05    3.13e+05
Sky            1.052e+05   2.42e+05      0.435      0.664    -3.7e+05     5.8e+05
Person         5372.9099   1.24e+04      0.435      0.664   -1.89e+04    2.96e+04
Rider           506.4711   1164.302      0.435      0.664   -1780.722    2793.665
Car            6.385e+04   1.47e+05      0.435      0.664   -2.24e+05    3.52e+05
Truck          1.737e+04   3.99e+04      0.435      0.664   -6.11e+04    9.58e+04
Bus            6131.9198   1.41e+04      0.435      0.664   -2.16e+04    3.38e+04
Train             2.4688      5.600      0.441      0.660      -8.533      13.470
Motorcycle     2071.5040   4761.885      0.435      0.664   -7282.901    1.14e+04
Bicycle        3227.4310   7419.090      0.435      0.664   -1.13e+04    1.78e+04
Unlabeled      2.342e+04   5.38e+04      0.435      0.664   -8.23e+04    1.29e+05
==============================================================================
Omnibus:                       10.274   Durbin-Watson:                   1.837
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               10.163
Skew:                           0.302   Prob(JB):                      0.00621
Kurtosis:                       2.722   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(hs_rd, clean_lo_rd)

#mid-road
run_reg(hs_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Humanscale   R-squared (uncentered):                   0.343
Model:                            OLS   Adj. R-squared (uncentered):              0.320
Method:                 Least Squares   F-statistic:                              14.64
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.17e-37
Time:                        17:10:41   Log-Likelihood:                         -667.33
No. Observations:                 552   AIC:                                      1373.
Df Residuals:                     533   BIC:                                      1455.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           3.492e+12   4.58e+12      0.762      0.446   -5.51e+12    1.25e+13
Sidewalk        1.86e+12   2.44e+12      0.762      0.446   -2.93e+12    6.65e+12
Building       6.317e+12   8.29e+12      0.762      0.446   -9.96e+12    2.26e+13
Wall            9.14e+11    1.2e+12      0.762      0.446   -1.44e+12    3.27e+12
Fence          9.834e+11   1.29e+12      0.762      0.446   -1.55e+12    3.52e+12
Pole           5.394e+11   7.08e+11      0.762      0.446   -8.51e+11    1.93e+12
Traffic Light  1.188e+11   1.56e+11      0.762      0.446   -1.87e+11    4.25e+11
Traffic Sign   2.322e+11   3.05e+11      0.762      0.446   -3.66e+11    8.31e+11
Vegetation      7.37e+12   9.67e+12      0.762      0.446   -1.16e+13    2.64e+13
Terrain        1.963e+12   2.58e+12      0.762      0.446    -3.1e+12    7.02e+12
Sky            6.343e+12   8.32e+12      0.762      0.446      -1e+13    2.27e+13
Person         2.211e+11    2.9e+11      0.762      0.446   -3.49e+11    7.91e+11
Rider          2.224e+10   2.92e+10      0.762      0.446   -3.51e+10    7.95e+10
Car            2.493e+12   3.27e+12      0.762      0.446   -3.93e+12    8.92e+12
Truck          5.291e+11   6.94e+11      0.762      0.446   -8.35e+11    1.89e+12
Bus            5.051e+11   6.63e+11      0.762      0.446   -7.97e+11    1.81e+12
Train          4.965e+09   6.51e+09      0.762      0.446   -7.83e+09    1.78e+10
Motorcycle     2.842e+10   3.73e+10      0.762      0.446   -4.48e+10    1.02e+11
Bicycle        9.687e+10   1.27e+11      0.762      0.446   -1.53e+11    3.47e+11
Unlabeled      2.177e+12   2.86e+12      0.762      0.446   -3.43e+12    7.79e+12
==============================================================================
Omnibus:                        3.371   Durbin-Watson:                   1.623
Prob(Omnibus):                  0.185   Jarque-Bera (JB):                3.405
Skew:                           0.166   Prob(JB):                        0.182
Kurtosis:                       2.807   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(hs_et, clean_lo_et)

#mid-emily-talen
run_reg(hs_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             Humanscale   R-squared (uncentered):                   0.275
Model:                            OLS   Adj. R-squared (uncentered):              0.262
Method:                 Least Squares   F-statistic:                              20.82
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    2.66e-63
Time:                        17:10:41   Log-Likelihood:                         -1408.0
No. Observations:                1119   AIC:                                      2856.
Df Residuals:                    1099   BIC:                                      2956.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           1.154e+04   7.05e+04      0.164      0.870   -1.27e+05     1.5e+05
Sidewalk       1.131e+04   6.91e+04      0.164      0.870   -1.24e+05    1.47e+05
Building       2.465e+04   1.51e+05      0.164      0.870   -2.71e+05     3.2e+05
Wall            619.9614   3787.690      0.164      0.870   -6811.959    8051.882
Fence          2835.7284   1.73e+04      0.164      0.870   -3.12e+04    3.68e+04
Pole           2245.1754   1.37e+04      0.164      0.870   -2.47e+04    2.92e+04
Traffic Light   633.7190   3872.365      0.164      0.870   -6964.345    8231.783
Traffic Sign   1215.3046   7423.877      0.164      0.870   -1.34e+04    1.58e+04
Vegetation     1.669e+04   1.02e+05      0.164      0.870   -1.83e+05    2.17e+05
Terrain        2852.7614   1.74e+04      0.164      0.870   -3.13e+04     3.7e+04
Sky            2.276e+04   1.39e+05      0.164      0.870    -2.5e+05    2.96e+05
Person         1509.4345   9221.131      0.164      0.870   -1.66e+04    1.96e+04
Rider            54.7993    335.286      0.163      0.870    -603.073     712.672
Car            7383.5746   4.51e+04      0.164      0.870   -8.11e+04    9.59e+04
Truck          1402.0216   8565.095      0.164      0.870   -1.54e+04    1.82e+04
Bus             851.8014   5204.045      0.164      0.870   -9359.185    1.11e+04
Train            27.2994    166.909      0.164      0.870    -300.197     354.795
Motorcycle       44.7358    273.451      0.164      0.870    -491.808     581.280
Bicycle         407.0684   2486.798      0.164      0.870   -4472.340    5286.477
Unlabeled       1.48e+04   9.04e+04      0.164      0.870   -1.63e+05    1.92e+05
==============================================================================
Omnibus:                        2.680   Durbin-Watson:                   1.483
Prob(Omnibus):                  0.262   Jarque-Bera (JB):                2.701
Skew:                           0.095   Prob(JB):                        0.259
Kurtosis:                       2.851   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

#### Disorder

In [ ]:
#low-sidewalk
run_reg(dis_sw, clean_lo_sw)

#mid-sidewalk
run_reg(dis_sw, clean_mid_sw)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               Disorder   R-squared (uncentered):                   0.507
Model:                            OLS   Adj. R-squared (uncentered):              0.488
Method:                 Least Squares   F-statistic:                              27.35
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    9.97e-69
Time:                        17:10:41   Log-Likelihood:                         -588.07
No. Observations:                 552   AIC:                                      1216.
Df Residuals:                     532   BIC:                                      1302.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           1.976e+05   1.63e+05      1.214      0.225   -1.22e+05    5.18e+05
Sidewalk       1.084e+05   8.93e+04      1.214      0.225   -6.71e+04    2.84e+05
Building       2.873e+05   2.37e+05      1.214      0.225   -1.78e+05    7.52e+05
Wall           4.222e+04   3.48e+04      1.214      0.225   -2.61e+04    1.11e+05
Fence          7.309e+04   6.02e+04      1.214      0.225   -4.52e+04    1.91e+05
Pole           2.067e+04    1.7e+04      1.214      0.225   -1.28e+04    5.41e+04
Traffic Light  5160.3422   4251.643      1.214      0.225   -3191.727    1.35e+04
Traffic Sign   1.185e+04   9759.397      1.214      0.225   -7326.416     3.1e+04
Vegetation     2.582e+05   2.13e+05      1.214      0.225    -1.6e+05    6.76e+05
Terrain        1.219e+05      1e+05      1.214      0.225   -7.54e+04    3.19e+05
Sky            2.261e+05   1.86e+05      1.214      0.225    -1.4e+05    5.92e+05
Person         1.154e+04   9511.155      1.214      0.225   -7140.001    3.02e+04
Rider          1088.2092    896.579      1.214      0.225    -673.061    2849.480
Car            1.372e+05   1.13e+05      1.214      0.225   -8.48e+04    3.59e+05
Truck          3.732e+04   3.07e+04      1.214      0.225   -2.31e+04    9.77e+04
Bus            1.317e+04   1.09e+04      1.214      0.225   -8148.623    3.45e+04
Train             5.2174      4.313      1.210      0.227      -3.254      13.689
Motorcycle     4450.6981   3666.925      1.214      0.225   -2752.731    1.17e+04
Bicycle        6934.1861   5713.126      1.214      0.225   -4288.867    1.82e+04
Unlabeled      5.032e+04   4.15e+04      1.214      0.225   -3.11e+04    1.32e+05
==============================================================================
Omnibus:                       11.952   Durbin-Watson:                   1.818
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               12.135
Skew:                           0.357   Prob(JB):                      0.00232
Kurtosis:                       3.138   Cond. No.                     2.32e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.54e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-road
run_reg(dis_rd, clean_lo_rd)

#mid-road
run_reg(dis_rd, clean_mid_rd)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               Disorder   R-squared (uncentered):                   0.172
Model:                            OLS   Adj. R-squared (uncentered):              0.143
Method:                 Least Squares   F-statistic:                              5.844
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.94e-13
Time:                        17:10:41   Log-Likelihood:                         -731.03
No. Observations:                 552   AIC:                                      1500.
Df Residuals:                     533   BIC:                                      1582.
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road            3.21e+12   5.14e+12      0.624      0.533   -6.89e+12    1.33e+13
Sidewalk       1.709e+12   2.74e+12      0.624      0.533   -3.67e+12    7.09e+12
Building       5.806e+12    9.3e+12      0.624      0.533   -1.25e+13    2.41e+13
Wall             8.4e+11   1.35e+12      0.624      0.533    -1.8e+12    3.48e+12
Fence          9.038e+11   1.45e+12      0.624      0.533   -1.94e+12    3.75e+12
Pole           4.958e+11   7.94e+11      0.624      0.533   -1.06e+12    2.06e+12
Traffic Light  1.091e+11   1.75e+11      0.624      0.533   -2.34e+11    4.53e+11
Traffic Sign   2.134e+11   3.42e+11      0.624      0.533   -4.58e+11    8.85e+11
Vegetation     6.773e+12   1.09e+13      0.624      0.533   -1.45e+13    2.81e+13
Terrain        1.804e+12   2.89e+12      0.624      0.533   -3.87e+12    7.48e+12
Sky            5.829e+12   9.34e+12      0.624      0.533   -1.25e+13    2.42e+13
Person         2.032e+11   3.26e+11      0.624      0.533   -4.36e+11    8.43e+11
Rider          2.044e+10   3.27e+10      0.624      0.533   -4.39e+10    8.48e+10
Car            2.291e+12   3.67e+12      0.624      0.533   -4.92e+12     9.5e+12
Truck          4.863e+11   7.79e+11      0.624      0.533   -1.04e+12    2.02e+12
Bus            4.642e+11   7.44e+11      0.624      0.533   -9.97e+11    1.93e+12
Train          4.563e+09   7.31e+09      0.624      0.533    -9.8e+09    1.89e+10
Motorcycle     2.612e+10   4.19e+10      0.624      0.533   -5.61e+10    1.08e+11
Bicycle        8.903e+10   1.43e+11      0.624      0.533   -1.91e+11    3.69e+11
Unlabeled          2e+12   3.21e+12      0.624      0.533    -4.3e+12     8.3e+12
==============================================================================
Omnibus:                       13.143   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               12.687
Skew:                           0.331   Prob(JB):                      0.00176
Kurtosis:                       2.664   Cond. No.                     7.70e+14
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.36e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
#low-emily-talen
run_reg(dis_et, clean_lo_et)

#mid-emily-talen
run_reg(dis_et, clean_mid_et)

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               Disorder   R-squared (uncentered):                   0.367
Model:                            OLS   Adj. R-squared (uncentered):              0.355
Method:                 Least Squares   F-statistic:                              31.84
Date:                Fri, 04 Sep 2020   Prob (F-statistic):                    1.39e-94
Time:                        17:10:41   Log-Likelihood:                         -1332.0
No. Observations:                1119   AIC:                                      2704.
Df Residuals:                    1099   BIC:                                      2804.
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Road           5.224e+04   6.58e+04      0.793      0.428    -7.7e+04    1.81e+05
Sidewalk       5.124e+04   6.46e+04      0.793      0.428   -7.55e+04    1.78e+05
Building       1.117e+05   1.41e+05      0.793      0.428   -1.64e+05    3.88e+05
Wall           2807.9722   3539.078      0.793      0.428   -4136.141    9752.085
Fence          1.284e+04   1.62e+04      0.793      0.428   -1.89e+04    4.46e+04
Pole           1.017e+04   1.28e+04      0.793      0.428    -1.5e+04    3.53e+04
Traffic Light  2870.9305   3618.196      0.793      0.428   -4228.421    9970.282
Traffic Sign   5503.8850   6936.598      0.793      0.428   -8106.586    1.91e+04
Vegetation     7.557e+04   9.52e+04      0.793      0.428   -1.11e+05    2.62e+05
Terrain        1.292e+04   1.63e+04      0.793      0.428    -1.9e+04    4.49e+04
Sky            1.031e+05    1.3e+05      0.793      0.428   -1.52e+05    3.58e+05
Person         6836.2782   8615.886      0.793      0.428   -1.01e+04    2.37e+04
Rider           248.5167    313.279      0.793      0.428    -366.175     863.208
Car            3.344e+04   4.21e+04      0.793      0.428   -4.93e+04    1.16e+05
Truck          6349.9386   8002.909      0.793      0.428   -9352.769    2.21e+04
Bus            3858.0755   4862.468      0.793      0.428   -5682.695    1.34e+04
Train           123.7611    155.953      0.794      0.428    -182.239     429.761
Motorcycle      202.6963    255.502      0.793      0.428    -298.631     704.023
Bicycle        1843.6103   2323.573      0.793      0.428   -2715.530    6402.750
Unlabeled      6.705e+04   8.45e+04      0.793      0.428   -9.88e+04    2.33e+05
==============================================================================
Omnibus:                       10.721   Durbin-Watson:                   1.713
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.756
Skew:                           0.237   Prob(JB):                      0.00462
Kurtosis:                       3.076   Cond. No.                     1.61e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.01e-11. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### VIF - Variance Inflation Factors for Mid-Level 
Attempt at solving issues with Multicollinearity


> *[VIFs](https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/) start at 1 and have no upper limit. A value of 1 indicates that there is no correlation between this independent variable and any others. VIFs between 1 and 5 suggest that there is a moderate correlation, but it is not severe enough to warrant corrective measures. VIFs greater than 5 represent critical levels of multicollinearity where the coefficients are poorly estimated, and the p-values are questionable.*



In [ ]:
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

lm = sm.OLS(all_pref, all_mid).fit()
variables = lm.model.exog
vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif = pd.DataFrame(vif)
vif.columns = ['VIFs']
vif
vif = vif.rename(index={0 : 'Road', 1 : 'Sidewalk', 2 :'Building', 3 :'Wall', 4 :'Fence', 5 :'Pole', 
                  6 : 'Traffic Light', 7: 'Traffic Sign', 8: 'Vegetation', 9 : 'Terrain', 10 : 'Sky',
                  11 : 'Person', 12 : 'Rider', 13 : 'Car', 14 : 'Truck', 15 : 'Bus', 16 : 'Train', 
                  17 : 'Motorcycle', 18 : 'Bicycle', 19 : 'Unlabeled'})
vif

,VIFs
Road,2.033546
Sidewalk,1.666718
Building,5.261987
Wall,1.028404
Fence,1.187411
Pole,1.378976
Traffic Light,1.206089
Traffic Sign,1.149362
Vegetation,4.838630
Terrain,1.489740


simple correlation matrix with builkding sky and veg

see correlation for Marc

### Stepwise OLS

### Low&Mid OLS 

#### Preference

#### Walkability

#### Imageability

#### Complexity

#### Enclosure

#### Transparency

#### Humanscale

#### Disorder

  Using cached https://files.pythonhosted.org/packages/b5/68/362dbd4bf0f692cdc1c9438fb6fea5be5ed1dab9b745ce81f6f8c7baed27/stepwisereg-0.1.0.tar.gz
ERROR: Could not find a version that satisfies the requirement statsmodel (from stepwisereg) (from versions: none)
ERROR: No matching distribution found for statsmodel (from stepwisereg)


## Cross Validation

# Figures